# 环境设置

In [7]:
                                                        

import pandas as pd
import os
import random

import torch

import sys



os.chdir(os.path.join(os.getcwd(), '..'))

if ".." not in sys.path:
    sys.path.insert(0, "..")

data_dir = '../data'



In [3]:
from Bio.Seq import Seq

In [8]:

os.getcwd()


'/maindata/data/user/user_wan/yul/ExpSPT'

In [3]:

1733*0.335


580.5550000000001

In [ ]:
%load_ext autoreload
%autoreload 2

# 数据

## proteinatlas

In [9]:


proteinatlas = pd.read_csv(os.path.join('data/proteinatlas', 'normal_ihc_data.tsv'), sep='\t')

proteinatlas.head()



,Gene,Gene name,Tissue,IHC tissue name,Cell type,Level,Reliability
0,ENSG00000000003,TSPAN6,Adipose tissue,Adipose tissue,adipocytes,Not detected,Approved
1,ENSG00000000003,TSPAN6,Adrenal gland,Adrenal gland,glandular cells,Not detected,Approved
2,ENSG00000000003,TSPAN6,Appendix,Appendix,glandular cells,Medium,Approved
3,ENSG00000000003,TSPAN6,Appendix,Appendix,lymphoid tissue,Not detected,Approved
4,ENSG00000000003,TSPAN6,Bone marrow,Bone marrow,hematopoietic cells,Not detected,Approved


In [11]:

proteinatlas['Level'].value_counts()
proteinatlas['Tissue'].value_counts()

proteinatlas['Cell type'].value_counts().shape

# proteinatlas.shape



Cell type
glandular cells              259573
squamous epithelial cells     65648
glial cells                   40409
neuronal cells                40198
fibroblasts                   39998
                              ...  
ductal cells                      9
cells in posterior                6
cells in dentate nucleus          4
ependymal cells                   4
atretic follicle                  1
Name: count, Length: 148, dtype: int64

In [12]:

proteinatlas_gtex = pd.read_csv(os.path.join('data/proteinatlas', 'rna_tissue_gtex.tsv'), sep='\t')

proteinatlas_gtex.head()


,Gene,Gene name,Tissue,TPM,pTPM,nTPM
0,ENSG00000000003,TSPAN6,adipose tissue,28.7,35.6,28.6
1,ENSG00000000003,TSPAN6,adrenal gland,15.2,18.4,17.6
2,ENSG00000000003,TSPAN6,amygdala,7.2,9.1,13.0
3,ENSG00000000003,TSPAN6,basal ganglia,7.8,9.6,12.4
4,ENSG00000000003,TSPAN6,breast,32.5,40.8,33.0


In [14]:
proteinatlas_gtex['Tissue'].value_counts()

proteinatlas_gtex.shape

(675206, 6)

In [41]:
common_set = set(proteinatlas['Tissue'].value_counts().reset_index()['index'].apply(lambda x: x.lower()).values) & set(proteinatlas_gtex['Tissue'].value_counts().index)

len(common_set)

union_set = set(proteinatlas['Tissue'].value_counts().reset_index()['index'].apply(lambda x: x.lower()).values) | set(proteinatlas_gtex['Tissue'].value_counts().index)

len(union_set)


62

In [26]:

set(proteinatlas['Tissue'].value_counts().index) & set(proteinatlas_gtex['Tissue'].value_counts().index)



set()

In [35]:

# 导入数据

CMA_expression_data = pd.read_csv(os.path.join('data/proteinatlas', 'expression_data.CMA.csv'), sep=',')

TMA_expression_data = pd.read_csv(os.path.join('data/proteinatlas', 'expression_data.TMA.csv'), sep=',')

IF_expression_data = pd.read_csv(os.path.join('data/proteinatlas', 'expression_data.IF.csv'), sep=',')


# 合并
whole_expression_data =  pd.DataFrame(columns=['hpa_id', 'ensembl_gene_id', 'celline', 'summary_expression_value'])

whole_expression_data = pd.concat([whole_expression_data, CMA_expression_data, TMA_expression_data, IF_expression_data])

whole_expression_data.head()




,hpa_id,ensembl_gene_id,celline,summary_expression_value,tissue,intensity,fraction
0,CAB000001,ENSG00000169083,A-431,negative,NaN,NaN,NaN
1,CAB000001,ENSG00000169083,A-549,negative,NaN,NaN,NaN
2,CAB000001,ENSG00000169083,AN3-CA,negative,NaN,NaN,NaN
3,CAB000001,ENSG00000169083,BEWO,negative,NaN,NaN,NaN
4,CAB000001,ENSG00000169083,CACO-2,negative,NaN,NaN,NaN


In [39]:

whole_expression_data['celline'].value_counts().shape


(45,)

In [42]:

# Tissues: 62
# Cellines: 45
# Seq: 101
# RNA_Exp:3
# Protein_Exp: 7
# special tokens:
    

62+45+101+3+7



218

## GTEx

In [8]:


GTEx = pd.read_csv(os.path.join('data/GTEx', 'GTEx_Analysis_v10_RNASeQCv2.4.2_gene_tpm.gct.tsv'), sep='\t')



In [9]:
GTEx.head()


,Name,Description,GTEX-1117F-0005-SM-HL9SH,GTEX-1117F-0011-R10b-SM-GI4VE,GTEX-1117F-0011-R11b-SM-GIN8R,GTEX-1117F-0011-R2b-SM-GI4VL,GTEX-1117F-0011-R3a-SM-GJ3PJ,GTEX-1117F-0011-R4b-SM-GI4VM,GTEX-1117F-0011-R5a-SM-GI4VW,GTEX-1117F-0011-R6a-SM-GI4VX,...,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2326-SM-GOQYU,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2526-SM-GOQZ3,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
0,ENSG00000223972.5,DDX11L1,0.00000,0.000000,0.000000,0.00000,0.0000,0.012397,0.00000,0.029395,...,0.000000,0.0000,0.000000,0.00000,0.00000,0.000000,0.000000,0.017791,0.020221,0.025713
1,ENSG00000227232.5,WASH7P,1.33343,3.579280,10.189300,2.96650,3.6828,3.599210,3.51443,4.286380,...,5.464380,2.3599,2.427010,3.87837,1.85892,2.383760,2.886100,5.095810,0.842465,2.209520
2,ENSG00000278267.1,MIR6859-1,0.00000,0.000000,0.000000,0.00000,0.0000,0.000000,0.00000,0.000000,...,0.000000,0.0000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
3,ENSG00000243485.5,MIR1302-2HG,0.00000,0.093825,0.034191,0.00000,0.0000,0.024750,0.00000,0.000000,...,0.062071,0.0000,0.086553,0.14685,0.00000,0.041073,0.053323,0.000000,0.000000,0.000000
4,ENSG00000237613.2,FAM138A,0.00000,0.000000,0.000000,0.01766,0.0000,0.000000,0.00000,0.000000,...,0.000000,0.0000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:


GTEx.shape



(51970, 19618)

## public cellines 

## Modern  MRL

## annotation data

In [3]:

from dataset.tokenizer import ntTokenizer, CDSTokenizer, Vocabulary

from dataset.data_prepare import _load_mRNA



In [3]:


ncRNA_fn = 'data/processed/mRNAfna/Human_gencode.v40.pc_transcripts.fa.norm.fa'


mRNAs = _load_mRNA(ncRNA_fn, compress=False)

print('{}:\t{}'.format(ncRNA_fn, len(mRNAs)))
    
    

data/processed/mRNAfna/Human_gencode.v40.pc_transcripts.fa.norm.fa:	88659


In [4]:
1271.5512001514435/60

21.19252000252406

In [4]:

RNA_data = pd.DataFrame(mRNAs, columns=['ids', 'seq', 'utr5', 'cds', 'utr3', 'aa', 'label'])


RNA_data.head()



,ids,seq,utr5,cds,utr3,aa,label
0,>ENST00000641515.2|ENSG00000186092.7|OTTHUMG00...,CCCAGATCTCTTCAGTTTTTATGCCTCATTCTGTGAAAATTGCTGT...,CCCAGATCTCTTCAGTTTTTATGCCTCATTCTGTGAAAATTGCTGT...,ATGAAGAAGGTAACTGCAGAGGCTATTTCCTGGAATGAATCAACGA...,ATCTTATATAACTGTGAGATTAATCTCAGATAATGACACAAAATAT...,MKKVTAEAISWNESTSETNNSMVTEFIFLGLSDSQELQTFLFMLFF...,None
1,>ENST00000426406.4|ENSG00000284733.2|OTTHUMG00...,ATGGATGGAGAGAATCACTCAGTGGTATCTGAGTTTTTGTTTCTGG...,,ATGGATGGAGAGAATCACTCAGTGGTATCTGAGTTTTTGTTTCTGG...,,MDGENHSVVSEFLFLGLTHSWEIQLLLLVFSSVLYVASITGNILIV...,None
2,>ENST00000332831.5|ENSG00000284662.2|OTTHUMG00...,ATGGATGGAGAGAATCACTCAGTGGTATCTGAGTTTTTGTTTCTGG...,,ATGGATGGAGAGAATCACTCAGTGGTATCTGAGTTTTTGTTTCTGG...,,MDGENHSVVSEFLFLGLTHSWEIQLLLLVFSSVLYVASITGNILIV...,None
3,>ENST00000616016.5|ENSG00000187634.13|OTTHUMG0...,GGCGGCGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTG...,GGCGGCGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTG...,ATGCCGGCGGTCAAGAAGGAGTTCCCGGGCCGCGAGGACCTGGCCC...,GGTTGCCGGGGGTAGGGGTGGGGCCACACAAATCTCCAGGAGCCAC...,MPAVKKEFPGREDLALALATFHPTLAALPLPPLPGYLAPLPAAAAL...,None
4,>ENST00000618323.5|ENSG00000187634.13|OTTHUMG0...,GGCGGCGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTG...,GGCGGCGGAGTCTCCCAAGTCCCCGCCGGGCGGGCGCGCGCCAGTG...,ATGCCGGCGGTCAAGAAGGAGTTCCCGGGCCGCGAGGACCTGGCCC...,GGTTGCCGGGGGTAGGGGTGGGGCCACACAAATCTCCAGGAGCCAC...,MPAVKKEFPGREDLALALATFHPTLAALPLPPLPGYLAPLPAAAAL...,None


In [10]:

RNA_aa_data = RNA_data.drop_duplicates(subset=['aa'])


In [11]:

RNA_aa_data.shape


(72415, 7)

In [20]:
os.getcwd()

'/maindata/data/user/user_wan/yul/bioSPT2'

In [25]:

with open('data/processed/mRNAfna/human_aa.fa', 'w') as f:
    for i,v in RNA_aa_data.iterrows():

        f.write(v['ids']+ '\n' + v['aa']+ '\n')
    

## 湿实验数据

### CDS 下限数据生成

In [1]:
from datasets import Dataset
from torch.utils.data import TensorDataset  
from datasets import load_dataset
from datasets import DatasetDict
from datasets import load_dataset, load_from_disk

from pandarallel import pandarallel
 
# Initialization
pandarallel.initialize(nb_workers=80, progress_bar=True)

import warnings
warnings.filterwarnings("ignore")

2024-06-06 15:20:06.749404: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-06 15:20:06.880818: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-06 15:20:07.639168: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-06 15:20:07.643621: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 15:20:09.625983: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

INFO: Pandarallel will run on 80 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [1]:
import shlex, subprocess

In [80]:
seq = 'MNDTEAI'
# p1 = subprocess.Popen(['echo', seq ], stdout=subprocess.PIPE, encoding='utf8')
p2= subprocess.Popen(["docker", "run", '-w', '/home/cyq/LinearDesign/','-it',  "jysgro/lineardesign:v1", 
                      "sh -c" + " '" + 'echo ' + seq + '| /home/cyq/LinearDesign/lineardesign --lambda 4' + "'"], stdout=subprocess.PIPE, shell=True, encoding='utf8')

# p1.communicate()[0].split('\n')[0]


Usage:	docker [OPTIONS] COMMAND

A self-sufficient runtime for containers

Options:
      --config string      Location of client config files (default
                           "/home/yul/.docker")
  -c, --context string     Name of the context to use to connect to the
                           daemon (overrides DOCKER_HOST env var and
                           default context set with "docker context use")
  -D, --debug              Enable debug mode
  -H, --host list          Daemon socket(s) to connect to
  -l, --log-level string   Set the logging level
                           ("debug"|"info"|"warn"|"error"|"fatal")
                           (default "info")
      --tls                Use TLS; implied by --tlsverify
      --tlscacert string   Trust certs signed only by this CA (default
                           "/home/yul/.docker/ca.pem")
      --tlscert string     Path to TLS certificate file (default
                           "/home/yul/.docker/cert.pem")
      --tlskey

In [81]:
seq = 'MNDTEAI'
p3= subprocess.Popen(["docker run jysgro/lineardesign:v1", 
                       "sh -c" + " '" + 'echo ' + seq + ' | /home/cyq/LinearDesign/lineardesign --lambda 4' + "'"], 
                     stdout=subprocess.PIPE, encoding='utf8', shell=True)


p3.communicate()



('', None)

In [71]:

subprocess.run(["docker run jysgro/lineardesign:v1", 
                       "sh -c" + " '" + 'echo ' + seq + ' | /home/cyq/LinearDesign/lineardesign --lambda 4' + "'"], capture_output=True,shell=True)

CompletedProcess(args=['docker run jysgro/lineardesign:v1', "sh -c 'echo MNDTEAI | /home/cyq/LinearDesign/lineardesign --lambda 4'"], returncode=0, stdout=b'', stderr=b'')

In [ ]:
templates_v3_NO38.parallel_apply(lambda x: genetic_evolution_parallel(x,  iterations=10, pop_size=10, 
                                                              outdir=os.path.join(os.getcwd(), 'data/processed/designed_seqs_v3_NO38')), axis=1)


In [ ]:
# CAI > 0.95 是一个不错的CDS

# docker pull jysgro/lineardesign:v1

# using linear design as basic cds designer

#### 汇总

In [4]:
os.getcwd()

'/data1/genomics'

In [215]:

from itertools import islice

# with open('/data1/genomics/yul/bioSPT2/scripts/BigLinearDesign_LAMBDA_merged.txt', 'r') as f:
with open('/data1/genomics/yul/bioSPT2/scripts/LinearDesign_LAMBDA_merged.txt', 'r') as f:
    lines = f.readlines()
    # lines=f.readlines()[0:15]
    
    cds_gens = []
    for line in lines:
        if line.startswith('>'):
            mRNA_seq = None
            mRNA_structure = None
            mfe=None
            cai=None
            ids = line.strip()
        elif line.startswith('mRNA sequence: '):
            # mRNA_seq = line.lstrip('mRNA sequence: ')
            mRNA_seq = line.split(':')[-1].strip()
        elif line.startswith('mRNA folding free energy:'):
            v_line= line.lstrip('mRNA folding free energy:').strip()
            mfe = v_line.split(' ')[0].strip()
            cai = v_line.split(' ')[-1].strip()
        elif line.startswith('mRNA structure:'):
                mRNA_structure = line.lstrip('mRNA structure:').strip()
        else:
            cds_gens.append([ids, mRNA_seq, mRNA_structure, mfe, cai])
            

In [197]:

cds_gens[0]

['>ENST00000641515.2|ENSG00000186092.7|OTTHUMG00000001094.4|OTTHUMT00000003223.4|OR4F5-201|OR4F5|2618|UTR5:1-60|CDS:61-1041|UTR3:1042-2618|',
 'UGAAGAAGGUGACCGCCGAGGCCAUCUCCUGGAACGAGAGCACCAGCGAGACCAACAACUCCAUGGUGACCGAGUUCAUCUUCCUGGGGCUGAGCGACUCCCAGGAGCUGCAGACCUUCCUGUUCAUGCUGUUCUUCGUGUUCUACGGGGGCAUCGUGUUCGGCAACCUGCUGAUCGUGAUCACCGUGGUGUCCGACAGCCACCUGCACAGCCCCAUGUACUUCCUGCUGGCCAACCUGAGCCUGAUCGACCUGAGCCUGUCCAGCGUGACAGCCCCCAAGAUGAUCACCGACUUCUUCAGCCAGAGGAAGGUGAUCAGCUUCAAGGGCUGUCUGGUGCAGAUCUUCCUGCUGCACUUCUUCGGGGGCAGCGAGAUGGUGAUCCUGAUCGCCAUGGGCUUCGAUCGGUACAUCGCCAUCUGCAAGCCCCUGCACUACACCACCAUCAUGUGCGGCAACGCCUGCGUGGGCAUCAUGGCCGUGACCUGGGGCAUCGGCUUCCUGCACAGCGUGAGCCAGCUGGCCUUCGCUGUGCACCUGCUGUUCUGCGGCCCCAACGAGGUGGACAGCUUCUACUGCGAUCUGCCUCGGGUGAUCAAGCUGGCCUGCACCGACACCUACAGGCUGGACAUCAUGGUGAUCGCCAACAGCGGGGUGCUGACCGUGUGCUCCUUCGUGCUGCUGAUCAUCAGCUACACCAUCAUCCUGAUGACCAUCCAGCACAGGCCCCUGGACAAGAGCAGCAAGGCCCUGAGCACCCUGACAGCCCACAUCACCGUGGUGCUGCUGUUCUUCGGGCCUUGCGUGUUCAUCUACGCCUGGCCCUUCCCCAUCAAGAGCCUGGACAAGUUCCUGGC

In [214]:

BigLinearDesign_LAMBDA_merged_CDS = pd.DataFrame(cds_gens, columns=['ids', 'mRNA_seq', 'mRNA_structure', 'mfe', 'cai'])
BigLinearDesign_LAMBDA_merged_CDS.shape
BigLinearDesign_LAMBDA_merged_CDS.head()


,ids,mRNA_seq,mRNA_structure,mfe,cai
0,>ENST00000641515.2|ENSG00000186092.7|OTTHUMG00...,AUGAAGAAGGUGACCGCCGAGGCCAUCUCCUGGAACGAGAGCACCA...,(((((...(((.((((..(((....((((((((..(....)..)))...,-420.60,0.987
1,>ENST00000458637.6|ENSG00000137474.23|OTTHUMG0...,AUGGUGAUCCUGCAGCAGGGCGACCACGUGUGGAUGGACCUGCGGC...,.((.((.(((((((.((((((((....(((((((.((...(((.((...,-3096.05,0.989
2,>ENST00000409619.6|ENSG00000137474.23|OTTHUMG0...,AUGGACCUGCGCCUGGGCCAGGAGUUCGACGUGCCCAUCGGCGCCG...,....((((((((((((.((.((((((..(((.((.(((((.........,-3078.55,0.989
3,>ENST00000660626.1|ENSG00000137474.23|OTTHUMG0...,AUGGAGGACCCAGCCGACCCCGCCGACCCCGCCCGGAUCCCCCAGC...,.....((.((((((((.(....(((...((((.(((((((((((((...,-108.90,0.990
4,>ENST00000409893.6|ENSG00000137474.23|OTTHUMG0...,CUGUUCGACAGGCACCUGUGUGUGCGCCAGCUGCGGUACUCCGGCA...,........(((((.((((.((((.((((.((((((((.(((((......,-824.00,0.991


In [212]:



BigLinearDesign_LAMBDA_merged_CDS['aa'] = BigLinearDesign_LAMBDA_merged_CDS['mRNA_seq'].apply(lambda x: ''.join(Seq(x).translate()))


BigLinearDesign_LAMBDA_merged_CDS['aa_valid'] = BigLinearDesign_LAMBDA_merged_CDS['aa'].apply(lambda x: True if x.find('*') == len(x)-1 else False)

BigLinearDesign_LAMBDA_merged_CDS['aa_valid'].sum()
BigLinearDesign_LAMBDA_merged_CDS.head()

,ids,mRNA_seq,mRNA_structure,mfe,cai,aa,aa_valid
0,>ENST00000641515.2|ENSG00000186092.7|OTTHUMG00...,AUGAAGAAGGUGACCGCCGAGGCCAUCUCCUGGAACGAGAGCACCA...,(((((...(((.((((..(((....((((((((..(....)..)))...,-420.60,0.987,MKKVTAEAISWNESTSETNNSMVTEFIFLGLSDSQELQTFLFMLFF...,True
1,>ENST00000458637.6|ENSG00000137474.23|OTTHUMG0...,AUGGUGAUCCUGCAGCAGGGCGACCACGUGUGGAUGGACCUGCGGC...,.((.((.(((((((.((((((((....(((((((.((...(((.((...,-3096.05,0.989,MVILQQGDHVWMDLRLGQEFDVPIGAVVKLCDSGQVQVVDDEDNEH...,True
2,>ENST00000409619.6|ENSG00000137474.23|OTTHUMG0...,AUGGACCUGCGCCUGGGCCAGGAGUUCGACGUGCCCAUCGGCGCCG...,....((((((((((((.((.((((((..(((.((.(((((.........,-3078.55,0.989,MDLRLGQEFDVPIGAVVKLCDSGQVQVVDDEDNEHWISPQNATHIK...,True
3,>ENST00000660626.1|ENSG00000137474.23|OTTHUMG0...,AUGGAGGACCCAGCCGACCCCGCCGACCCCGCCCGGAUCCCCCAGC...,.....((.((((((((.(....(((...((((.(((((((((((((...,-108.90,0.990,MEDPADPADPARIPQPGKWALVRSLNQAFKTYAVLPGDHVWMDLRL...,False
4,>ENST00000409893.6|ENSG00000137474.23|OTTHUMG0...,CUGUUCGACAGGCACCUGUGUGUGCGCCAGCUGCGGUACUCCGGCA...,........(((((.((((.((((.((((.((((((((.(((((......,-824.00,0.991,LFDRHLCVRQLRYSGMMETIRIRRAGYPIRYSFVEFVERYRVLLPG...,True


In [218]:
BigLinearDesign_LAMBDA_merged_CDS.shape

(2278, 5)

In [216]:

LinearDesign_LAMBDA_merged_CDS = pd.DataFrame(cds_gens, columns=['ids', 'mRNA_seq', 'mRNA_structure', 'mfe', 'cai'])


In [219]:

LinearDesign_LAMBDA_merged_CDS.shape
LinearDesign_LAMBDA_merged_CDS.head()


,ids,mRNA_seq,mRNA_structure,mfe,cai
0,>ENST00000641515.2|ENSG00000186092.7|OTTHUMG00...,AUGAAGAAGGUGACCGCCGAGGCCAUCUCCUGGAACGAGAGCACCA...,(((((...(((.((((..(((....((((((((..(....)..)))...,-420.60,0.987
1,>ENST00000426406.4|ENSG00000284733.2|OTTHUMG00...,AUGGACGGCGAGAACCACAGCGUGGUGUCCGAGUUCCUGUUCCUGG...,....((((((.(((((.((((((((.(((..((((((.......))...,-416.90,0.986
2,>ENST00000616016.5|ENSG00000187634.13|OTTHUMG0...,AUGCCAGCCGUGAAGAAGGAGUUCCCCGGGCGGGAGGAUCUGGCCC...,..(((((..((..((((((.(((((((((((((((((.(((((((....,-1578.40,0.975
3,>ENST00000618323.5|ENSG00000187634.13|OTTHUMG0...,AUGCCAGCCGUGAAGAAGGAGUUCCCCGGGCGGGAGGAUCUGGCCC...,..(((((..((..((((((.(((((((((((((((((.(((((((....,-1584.70,0.975
4,>ENST00000437963.5|ENSG00000187634.13|OTTHUMG0...,AUGAGCAAGGGCAUCCUGCAGGUGCACCCCCCCAUCUGCGACUGUC...,........(((((...((((((((........)))))))).(((((...,-173.00,0.986


In [220]:

LinearDesign_LAMBDA_merged_CDS_gens = pd.concat([LinearDesign_LAMBDA_merged_CDS, BigLinearDesign_LAMBDA_merged_CDS], axis=0)

LinearDesign_LAMBDA_merged_CDS_gens.shape

(5139, 5)

In [221]:

LinearDesign_LAMBDA_merged_CDS_gens.isnull().sum()

# LinearDesign_LAMBDA_merged_CDS['mRNA_seq'].value_counts()
LinearDesign_LAMBDA_merged_CDS_gens_unique = LinearDesign_LAMBDA_merged_CDS_gens.drop_duplicates(subset=['ids','mRNA_seq'], keep='first', inplace=False)



In [222]:

LinearDesign_LAMBDA_merged_CDS_gens_unique.to_csv('/data1/genomics/yul/bioSPT2/data/interiem/processed/LinearDesign_LAMBDA_merged_CDS_gens_unique.csv', index=False)

In [44]:

LinearDesign_LAMBDA_merged_CDS_gens_unique['ids'].value_counts().shape

(1320,)

In [223]:


LinearDesign_LAMBDA_merged_CDS_gens_unique = pd.read_csv('/data1/genomics/yul/bioSPT2/data/interiem/processed/LinearDesign_LAMBDA_merged_CDS_gens_unique.csv')

LinearDesign_LAMBDA_merged_CDS_gens_unique.head()

,ids,mRNA_seq,mRNA_structure,mfe,cai
0,>ENST00000641515.2|ENSG00000186092.7|OTTHUMG00...,AUGAAGAAGGUGACCGCCGAGGCCAUCUCCUGGAACGAGAGCACCA...,(((((...(((.((((..(((....((((((((..(....)..)))...,-420.6,0.987
1,>ENST00000426406.4|ENSG00000284733.2|OTTHUMG00...,AUGGACGGCGAGAACCACAGCGUGGUGUCCGAGUUCCUGUUCCUGG...,....((((((.(((((.((((((((.(((..((((((.......))...,-416.9,0.986
2,>ENST00000616016.5|ENSG00000187634.13|OTTHUMG0...,AUGCCAGCCGUGAAGAAGGAGUUCCCCGGGCGGGAGGAUCUGGCCC...,..(((((..((..((((((.(((((((((((((((((.(((((((....,-1578.4,0.975
3,>ENST00000618323.5|ENSG00000187634.13|OTTHUMG0...,AUGCCAGCCGUGAAGAAGGAGUUCCCCGGGCGGGAGGAUCUGGCCC...,..(((((..((..((((((.(((((((((((((((((.(((((((....,-1584.7,0.975
4,>ENST00000437963.5|ENSG00000187634.13|OTTHUMG0...,AUGAGCAAGGGCAUCCUGCAGGUGCACCCCCCCAUCUGCGACUGUC...,........(((((...((((((((........)))))))).(((((...,-173.0,0.986


In [179]:
from Bio.Seq import Seq


In [180]:

LinearDesign_LAMBDA_merged_CDS_gens_unique['aa'] = LinearDesign_LAMBDA_merged_CDS_gens_unique['mRNA_seq'].apply(lambda x: ''.join(Seq(x).translate()))



/home/yul/softwares/anaconda3/envs/ai/lib/python3.8/site-packages/Bio/Seq.py:2334: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [183]:

LinearDesign_LAMBDA_merged_CDS_gens_unique['aa'].head()

LinearDesign_LAMBDA_merged_CDS_gens_unique.head()

,ids,mRNA_seq,mRNA_structure,mfe,cai,aa
0,>ENST00000641515.2|ENSG00000186092.7|OTTHUMG00...,UGAAGAAGGUGACCGCCGAGGCCAUCUCCUGGAACGAGAGCACCAG...,(((((...(((.((((..(((....((((((((..(....)..)))...,-420.6,0.987,*RR*PPRPSPGTRAPARPTTPW*PSSSSWG*ATPRSCRPSCSCCSS...
1,>ENST00000426406.4|ENSG00000284733.2|OTTHUMG00...,UGGACGGCGAGAACCACAGCGUGGUGUCCGAGUUCCUGUUCCUGGG...,....((((((.(((((.((((((((.(((..((((((.......))...,-416.9,0.986,WTARTTAWCPSSCSWD*PTAGRSSCCCWCSAACCTWPASPATS*SC...
2,>ENST00000616016.5|ENSG00000187634.13|OTTHUMG0...,UGCCAGCCGUGAAGAAGGAGUUCCCCGGGCGGGAGGAUCUGGCCCU...,..(((((..((..((((((.(((((((((((((((((.(((((((....,-1578.4,0.975,CQP*RRSSPGGRIWPWPWPPSTPPWPPCPCPPCLATWPPCQPPPPC...
3,>ENST00000618323.5|ENSG00000187634.13|OTTHUMG0...,UGCCAGCCGUGAAGAAGGAGUUCCCCGGGCGGGAGGAUCUGGCCCU...,..(((((..((..((((((.(((((((((((((((((.(((((((....,-1584.7,0.975,CQP*RRSSPGGRIWPWPWPPSTPPWPPCPCPPCLATWPLCQPPPPC...
4,>ENST00000437963.5|ENSG00000187634.13|OTTHUMG0...,UGAGCAAGGGCAUCCUGCAGGUGCACCCCCCCAUCUGCGACUGUCC...,........(((((...((((((((........)))))))).(((((...,-173.0,0.986,*ARASCRCTPPSATVLAAGSALP*TGAAWPTSGPWPCPPPGT*RRS...


## UTR 下限数据

In [ ]:

去多细胞多组学数据中寻找

In [48]:
os.getcwd()

'/data1/genomics'

In [2]:
# 加载公共注释数据
dataset_dir = '/data1/genomics/yul/bioSPT2/data/interiem/annotation'

dominant_Txs_select_rank1_top = pd.read_csv(os.path.join(dataset_dir, 'genecode_human_dominant_Txs_select_rank1_top_processed.tsv'), sep='\t')

dominant_Txs_select_rank1_top.head()


human_ts = pd.read_csv(os.path.join(dataset_dir, 'genecode_human_annotation_dominant_transcript_cdna.tsv'), sep='\t')

human_ts.head()


,Transcript,gene_id,name,seq,transcript_id,gene_id_raw,gene_name,UTR5_regs,CDS_regs,UTR3_regs,UTR5,CDS,UTR3,Esemble
0,ENST00000373020,ENSG00000000003.15,>ENST00000373020.9|ENSG00000000003.15|OTTHUMG0...,AGTTGTGGACGCTCGTAAGTTTTCGGCAGTTTCCGGGGAGACTCGG...,ENST00000373020.9,ENST00000373020.9,TSPAN6,UTR5:1-112,CDS:113-850,UTR3:851-3768,AGTTGTGGACGCTCGTAAGTTTTCGGCAGTTTCCGGGGAGACTCGG...,ATGGCGTCCCCGTCTCGGAGACTGCAGACTAAACCAGTCATTACTT...,CCCAATGTATCTGTGGGCCTATTCCTCTCTACCTTTAAGGACATTT...,ENSG00000000003
1,ENST00000373031,ENSG00000000005.6,>ENST00000373031.5|ENSG00000000005.6|OTTHUMG00...,AGCCGACTCACTTGCAACTCCACCTCAGCAGTGGTCTCTCAGTCCT...,ENST00000373031.5,ENST00000373031.5,TNMD,UTR5:1-83,CDS:84-1037,UTR3:1038-1205,AGCCGACTCACTTGCAACTCCACCTCAGCAGTGGTCTCTCAGTCCT...,ATGGCAAAGAATCCTCCAGAGAATTGTGAAGACTGTCACATTCTAA...,TAGGAGGTTTGAGCTCAAATGCTTAAACTGCTGGCAACATATAATA...,ENSG00000000005
2,ENST00000371588,ENSG00000000419.14,>ENST00000371588.10|ENSG00000000419.14|OTTHUMG...,AGTTCCGCCATGGCCTCCTTGGAAGTCAGTCGTAGTCCTCGCAGGT...,ENST00000371588.10,ENST00000371588.10,DPM1,UTR5:1-9,CDS:10-792,UTR3:793-1054,AGTTCCGCC,ATGGCCTCCTTGGAAGTCAGTCGTAGTCCTCGCAGGTCTCGGCGGG...,AAGAAAGATACTCATTTATAGTTACGTTCATTTCAGGTTAAACATG...,ENSG00000000419
3,ENST00000367772,ENSG00000000457.14,>ENST00000367772.8|ENSG00000000457.14|OTTHUMG0...,TGTCCCGTTTCCGGACCCGTCTCTATGGTGTAGGAGAAACCCGGCC...,ENST00000367772.8,ENST00000367772.8,SCYL3,UTR5:1-222,CDS:223-2451,UTR3:2452-3090,TGTCCCGTTTCCGGACCCGTCTCTATGGTGTAGGAGAAACCCGGCC...,ATGGGATCAGAGAACAGTGCTTTAAAGAGCTATACACTGAGAGAAC...,CAATAGATGTGAGTTAAACTTTAGGAAAAAGGATTCCCTTTTTTTA...,ENSG00000000457
4,ENST00000413811,ENSG00000000460.17,>ENST00000413811.3|ENSG00000000460.17|OTTHUMG0...,ACATTCTTGATTGGTCTCATGCCTTTGTGGTTGTAAATGTGCCTGG...,ENST00000413811.3,ENST00000413811.3,C1orf112,UTR5:1-1018,CDS:1019-2611,UTR3:2612-2661,ACATTCTTGATTGGTCTCATGCCTTTGTGGTTGTAAATGTGCCTGG...,ATGCAGGTGGTTTTGGACAGTAAATTAGACCTGCCATGTGAACTGC...,AACTTATCACTAGGCAGAACTGGGTTTGATGCTTTGTCAACTGAAA...,ENSG00000000460


In [7]:
# 多组学多细胞系数据

all_genes_omics = pd.read_csv('/data1/genomics/yul/RNA_optimization/ExpUTR5/data/external/cleandata/ALL_gene_omics.csv', sep=',')

all_genes_omics = all_genes_omics.set_index('ensembl_gene_id')

all_genes_omics.head()

,iBAQ A549_exp,iBAQ HEK293_exp,iBAQ HeLa_exp,iBAQ K562_exp,iBAQ_TransEffect_A549_exp,iBAQ_TransEffect_HEK293_exp,iBAQ_TransEffect_HeLa_exp,iBAQ_TransEffect_K562_exp,iBAQ_TransEffect_A549_rank,iBAQ_TransEffect_HEK293_rank,...,K562_GR_2017_rank,K562_elife_rank,K562_timeLapse_rank,ESC_GR_2017_label,Hela_GSE102113_label,K562_GR_2017_label,K562_elife_label,K562_timeLapse_label,consistent_label_HalfLife,consistent_label_all
ensembl_gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,6.527933,6.796727,5.517671,NaN,5.134571,5.195596,4.181992,NaN,1074.0,2159.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
ENSG00000000419,6.989328,6.970460,6.817138,6.404391,5.275929,5.256304,4.987699,4.315847,756.0,1958.0,...,1926.0,5030.0,1362.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN
ENSG00000000457,4.331123,5.157944,3.871246,NaN,3.588284,4.523821,3.200597,NaN,5498.0,4664.0,...,9413.0,8183.0,NaN,-1.0,NaN,-1.0,-1.0,NaN,NaN,NaN
ENSG00000000460,5.157366,5.762855,4.583522,5.264936,3.871606,4.495252,3.234262,3.624279,4839.0,4755.0,...,5547.0,4374.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN
ENSG00000000938,NaN,4.211548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:

celllines_cols = [item  for item in all_genes_omics.columns if 'label' in item and 'consistent_label' not in item]

celllines_high_genes, celllines_counts, cellines_top_10_genes, cellines_top_20_genes = [], [], [], []
for col in celllines_cols:
    celllines_high_genes.append(list(all_genes_omics[col][all_genes_omics[col]==1].index))
    cellines_top_10_genes.append(list(all_genes_omics[all_genes_omics[col]==1
                                                     ][col.replace('label', 'exp')].sort_values(ascending=False).head(10).index))
    cellines_top_20_genes.append(list(all_genes_omics[all_genes_omics[col]==1
                                                     ][col.replace('label', 'exp')].sort_values(ascending=False).head(20).index))
    
    celllines_counts.append(all_genes_omics[col][all_genes_omics[col]==1].shape[0])
    
cellines_pd = pd.DataFrame()

cellines_pd['dataset']  = celllines_cols
cellines_pd['data size']  = celllines_counts
cellines_pd['top 10 genes']  = cellines_top_10_genes
cellines_pd['top 20 genes']  = cellines_top_20_genes

cellines_pd


,dataset,data size,top 10 genes,top 20 genes
0,iBAQ_TransEffect_A549_label,1698,"[ENSG00000203811, ENSG00000203852, ENSG0000018...","[ENSG00000203811, ENSG00000203852, ENSG0000018..."
1,iBAQ_TransEffect_HEK293_label,1978,"[ENSG00000203811, ENSG00000203852, ENSG0000018...","[ENSG00000203811, ENSG00000203852, ENSG0000018..."
2,iBAQ_TransEffect_HeLa_label,1805,"[ENSG00000204389, ENSG00000158373, ENSG0000027...","[ENSG00000204389, ENSG00000158373, ENSG0000027..."
3,iBAQ_TransEffect_K562_label,1571,"[ENSG00000203811, ENSG00000183598, ENSG0000020...","[ENSG00000203811, ENSG00000183598, ENSG0000020..."
4,TransEffect_Hela_Nagaraj_label,1247,"[ENSG00000254618, ENSG00000243678, ENSG0000013...","[ENSG00000254618, ENSG00000243678, ENSG0000013..."
5,TransEffect_Hela_Zecha_label,1485,"[ENSG00000131323, ENSG00000131171, ENSG0000010...","[ENSG00000131323, ENSG00000131171, ENSG0000010..."
6,A549-SRR611121-SRR3286543-zhanggong_label,3015,"[ENSG00000277027, ENSG00000284459, ENSG0000025...","[ENSG00000277027, ENSG00000284459, ENSG0000025..."
7,hela-SRR8443314-SRR8443308_label,2914,"[ENSG00000200169, ENSG00000238825, ENSG0000027...","[ENSG00000200169, ENSG00000238825, ENSG0000027..."
8,HEK293-SRR1795409_10_label,3557,"[ENSG00000178977, ENSG00000138606, ENSG0000011...","[ENSG00000178977, ENSG00000138606, ENSG0000011..."
9,ESC_GR_2017_label,2367,"[ENSG00000105618, ENSG00000172340, ENSG0000010...","[ENSG00000105618, ENSG00000172340, ENSG0000010..."


In [5]:
from itertools import chain

all_cellines_top_10_genes = set(list(chain.from_iterable(cellines_top_10_genes)))
all_cellines_top_20_genes = set(list(chain.from_iterable(cellines_top_20_genes)))
print(f" top 10 total: {len(all_cellines_top_10_genes)}")

print(f" top 20 total: {len(all_cellines_top_20_genes)}")


 top 10 total: 113
 top 20 total: 216


In [ ]:

all_cells_omics_mRNA = os.path.join('/home/yul/project/nas_project/bioSPT/data/processed/mRNAfna', 'all_cells_omics_mRNA.train.fa')
 

In [64]:

# all_genes_omics

all_genes_omics[celllines_cols]

all_genes_omics[all_genes_omics[col]==1][col.replace('label', 'exp')].sort_values(ascending=False).head(10).index

Index(['ENSG00000196565', 'ENSG00000142534', 'ENSG00000137571',
       'ENSG00000142168', 'ENSG00000137818', 'ENSG00000168653',
       'ENSG00000182774', 'ENSG00000118181', 'ENSG00000231500',
       'ENSG00000084207'],
      dtype='object', name='ensembl_gene_id')

In [9]:
# 所有细胞系共用
all_genes_omics_consistent_genes = all_genes_omics[(all_genes_omics[celllines_cols] == 1).all(axis=1)].index

all_cellines_top_10_genes & set(all_genes_omics_consistent_genes)


set()

In [10]:

UTR_genes = (all_cellines_top_20_genes |  set(all_genes_omics_consistent_genes))


UTR_genes

{'ENSG00000000003',
 'ENSG00000013306',
 'ENSG00000013583',
 'ENSG00000060762',
 'ENSG00000067177',
 'ENSG00000069849',
 'ENSG00000075945',
 'ENSG00000079819',
 'ENSG00000083457',
 'ENSG00000084207',
 'ENSG00000084754',
 'ENSG00000087086',
 'ENSG00000087191',
 'ENSG00000089280',
 'ENSG00000101082',
 'ENSG00000101152',
 'ENSG00000101444',
 'ENSG00000102007',
 'ENSG00000102178',
 'ENSG00000102317',
 'ENSG00000103202',
 'ENSG00000103275',
 'ENSG00000103534',
 'ENSG00000105193',
 'ENSG00000105583',
 'ENSG00000105618',
 'ENSG00000108294',
 'ENSG00000108298',
 'ENSG00000109107',
 'ENSG00000109846',
 'ENSG00000110107',
 'ENSG00000110651',
 'ENSG00000111229',
 'ENSG00000111911',
 'ENSG00000112335',
 'ENSG00000112378',
 'ENSG00000114023',
 'ENSG00000114942',
 'ENSG00000115112',
 'ENSG00000115541',
 'ENSG00000115685',
 'ENSG00000116209',
 'ENSG00000116288',
 'ENSG00000116455',
 'ENSG00000117395',
 'ENSG00000117419',
 'ENSG00000117450',
 'ENSG00000117592',
 'ENSG00000118181',
 'ENSG00000119396',


In [11]:

len(UTR_genes)

224

In [20]:

UTR_genes_human_ts = human_ts.set_index('Esemble').reindex(UTR_genes)


UTR_genes_human_ts_clean = UTR_genes_human_ts[
    ~UTR_genes_human_ts["UTR5"].isnull() & ~UTR_genes_human_ts["UTR3"].isnull() ]


In [16]:
import gzip

# with gzip.open('/data1/genomics/yul/bioSPT2/data/interiem/moderna_polysome/GSM3130435_egfp_unmod_1.csv.gz', 'r') as f:
    
egfp_unmod_1 = pd.read_csv('/data1/genomics/yul/bioSPT2/data/interiem/moderna_polysome/GSM3130435_egfp_unmod_1.csv.gz', compression='gzip', sep=',', index_col=0)

egfp_unmod_1.head()

,utr,0,1,2,3,4,5,6,7,8,...,r5,r6,r7,r8,r9,r10,r11,r12,r13,rl
120605,CCACTCGATTAACATGTTAACAACATACTCGTCCGGCCGATCAGCG...,0.000137,0.000109,0.000057,0.000033,0.000015,0.000016,0.000009,0.000009,0.000008,...,0.033645,0.020343,0.019850,0.017988,0.019283,0.033252,0.033252,0.027581,0.035877,3.039939
11605,CAAATCATGTGCAGCCCTGGCGACCGTACTGCGGTACAAGAAAGTA...,0.000067,0.000070,0.000065,0.000048,0.000023,0.000020,0.000011,0.000011,0.000010,...,0.049711,0.028540,0.027431,0.024766,0.024241,0.039457,0.039457,0.039840,0.038785,3.895109
128224,GTTATACTAGAAGAAACTTGAGATTATGGAGCAGTCCGTCAAGGAC...,0.000088,0.000081,0.000059,0.000035,0.000017,0.000016,0.000009,0.000009,0.000009,...,0.042436,0.022460,0.025015,0.024275,0.021591,0.028353,0.028353,0.028963,0.041985,3.334524
239107,CTTAGACAAAAACAACGCGCTTTCCAGTATGCGGAGCCTTGACGGT...,0.000078,0.000071,0.000060,0.000038,0.000025,0.000016,0.000010,0.000009,0.000009,...,0.041241,0.025793,0.022751,0.022632,0.026617,0.038302,0.038302,0.032788,0.031043,3.575082
59082,GTATCAAATCACGGCCAACCCGACGGAGTACCCCGCGTCGATGGTC...,0.000044,0.000045,0.000050,0.000051,0.000033,0.000026,0.000015,0.000012,0.000011,...,0.071306,0.040719,0.033335,0.030367,0.033080,0.051449,0.051449,0.046052,0.036447,4.593712


In [28]:

UTR_genes_human_ts_clean_all = pd.concat([UTR_genes_human_ts_clean, pd.DataFrame(egfp_unmod_1.sort_values(by='rl', ascending=False).head(50)['utr']).rename(columns={'utr':'UTR5'})], axis=0)

UTR_genes_human_ts_clean_all.to_csv('/data1/genomics/yul/bioSPT2/data/interiem/processed/UTR_high_genes_human_ts_clean_all.csv', sep=',')


In [29]:

UTR_genes_human_ts_clean_all


,Transcript,gene_id,name,seq,transcript_id,gene_id_raw,gene_name,UTR5_regs,CDS_regs,UTR3_regs,UTR5,CDS,UTR3
ENSG00000087086,ENST00000331825,ENSG00000087086.15,>ENST00000331825.11|ENSG00000087086.15|OTTHUMG...,GCAGTTCGGCGGTCCCGCGGGTCTGTCTCTTGCTTCAACAGTGTTT...,ENST00000331825.11,ENST00000331825.11,FTL,UTR5:1-199,CDS:200-727,UTR3:728-871,GCAGTTCGGCGGTCCCGCGGGTCTGTCTCTTGCTTCAACAGTGTTT...,ATGAGCTCCCAGATTCGTCAGAATTATTCCACCGACGTGGAGGCAG...,GAGCCTTCTGAGCCCAGCGACTTCTGAAGGGCCCCTTGCAAAGTAA...
ENSG00000138606,ENST00000458022,ENSG00000138606.20,>ENST00000458022.6|ENSG00000138606.20|OTTHUMG0...,GAGTTGCCACCTGGGTGCCCATGGGCACTGCAGGCTAGAGCTGAGG...,ENST00000458022.6,ENST00000458022.6,SHF,UTR5:1-209,CDS:210-929,UTR3:930-1163,GAGTTGCCACCTGGGTGCCCATGGGCACTGCAGGCTAGAGCTGAGG...,ATGGAGCCCTATGAGGCTCAAAAGATGATGGCCGAGATCCGGGGCT...,ATGTGAAGCCAGGGCACTGTGATAGACCTGTACCCAGCCCTGTGCC...
ENSG00000105618,ENST00000321030,ENSG00000105618.14,>ENST00000321030.9|ENSG00000105618.14|OTTHUMG0...,GGTGAGCGACTAACGCTAGAAACAGTGGTGCGCGGAGAGGAGAGGC...,ENST00000321030.9,ENST00000321030.9,PRPF31,UTR5:1-52,CDS:53-1552,UTR3:1553-1833,GGTGAGCGACTAACGCTAGAAACAGTGGTGCGCGGAGAGGAGAGGC...,ATGTCTCTGGCAGATGAGCTCTTAGCTGATCTCGAAGAGGCAGCAG...,ATGACTGCGTGTGTCCAAGGTGGCTTCCCACTGAAGGGACACAGAG...
ENSG00000164111,ENST00000296511,ENSG00000164111.15,>ENST00000296511.10|ENSG00000164111.15|OTTHUMG...,AGTCTAGGTGCAGCTGCCGGATCCTTCAGCGTCTGCATCTCGGCGT...,ENST00000296511.10,ENST00000296511.10,ANXA5,UTR5:1-153,CDS:154-1116,UTR3:1117-1638,AGTCTAGGTGCAGCTGCCGGATCCTTCAGCGTCTGCATCTCGGCGT...,ATGGCACAGGTTCTCAGAGGCACTGTGACTGACTTCCCTGGATTTG...,CGTGTCACGGGGAAGAGCTCCCTGCTGTGTGCCTGCACCACCCCAC...
ENSG00000278637,ENST00000617569,ENSG00000278637.2,>ENST00000617569.2|ENSG00000278637.2|OTTHUMG00...,ACTTGCTCTTGGTTCACTTCTTGGGAAGTCATGTCTGGACGTGGTA...,ENST00000617569.2,ENST00000617569.2,H4C1,UTR5:1-30,CDS:31-342,UTR3:343-402,ACTTGCTCTTGGTTCACTTCTTGGGAAGTC,ATGTCTGGACGTGGTAAGGGCGGGAAGGGTTTGGGTAAGGGGGGTG...,GGTTGCTGATTTCTCCACAGCTTGCATTTCTGAACCAAAGGCCCTT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
325682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TAATATCATTCGTCTACACAACTACGATATGGAAGCTATTTCGCAG...,NaN,NaN
325969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCGGGAGATGTACTGGGAGAGATTTCCTTCAAGCGTAGAAGATCAT...,NaN,NaN
325933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CATCTATGCATAACGATATAAAACATTAGAGTCAACCTTTAAGCAG...,NaN,NaN
325719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GGTGCCGCACAACGCCGGTAAATAGTAGTAGGGTCGTGCACATCAC...,NaN,NaN


## mRNA 构造

In [224]:

# load from file

LinearDesign_LAMBDA_merged_CDS_gens_unique = pd.read_csv('/data1/genomics/yul/bioSPT2/data/interiem/processed/LinearDesign_LAMBDA_merged_CDS_gens_unique.csv')

LinearDesign_LAMBDA_merged_CDS_gens_unique.shape


raw_genes_ids = LinearDesign_LAMBDA_merged_CDS_gens_unique.drop_duplicates(subset=['ids'], keep='first', inplace=False)['ids'].apply(lambda x: x.split('|')[1].split('.')[0]).value_counts().index

raw_genes_ids
raw_genes_mRNAs = human_ts.set_index('Esemble').reindex(raw_genes_ids).reset_index()


In [225]:

raw_high_genes_mRNAs = UTR_genes_human_ts_clean_all[
    
    ~UTR_genes_human_ts_clean_all['UTR5'].isnull() &
    ~UTR_genes_human_ts_clean_all['UTR3'].isnull() &
    ~UTR_genes_human_ts_clean_all['CDS'].isnull()  
                             ]



In [80]:
# cds、utr配对

# CDS

UTR_genes_human_ts_clean_all['Transcript'].apply(lambda x: False if isinstance(x, str)  else True)
# UTR_genes_human_ts_clean_all['Transcript_MRL']


UTR_genes_human_ts_clean_all[
UTR_genes_human_ts_clean_all['Transcript'].apply(lambda x: False if isinstance(x, str)  else True)].reset_index().index

UTR_genes_human_ts_clean_all.loc[UTR_genes_human_ts_clean_all['Transcript'].apply(lambda x: False if isinstance(x, str)  else True)].index = list(range(50))

UTR_genes_human_ts_clean_all['Transcript'] = UTR_genes_human_ts_clean_all.loc[~UTR_genes_human_ts_clean_all['Transcript'].apply(lambda x: False if isinstance(x, str)  else True)]['Transcript'].values.tolist() + [f'MRL_TOP{a}' for a in range(1, 51)]

UTR_genes_human_ts_clean_all['Transcript']


ENSG00000087086    ENST00000331825
ENSG00000138606    ENST00000458022
ENSG00000105618    ENST00000321030
ENSG00000164111    ENST00000296511
ENSG00000278637    ENST00000617569
                        ...       
325682                   MRL_TOP46
325969                   MRL_TOP47
325933                   MRL_TOP48
325719                   MRL_TOP49
325728                   MRL_TOP50
Name: Transcript, Length: 215, dtype: object

In [137]:

agg_LinearDesign_LAMBDA_merged_CDS_gens_unique

,mRNA_seq,mRNA_structure,mfe,cai
ids,,,,
>ENST00000000233.10|ENSG00000004059.11|OTTHUMG00000023246.7|OTTHUMT00000059567.3|ARF5-201|ARF5|1032|UTR5:1-88|CDS:89-631|UTR3:632-1032|,[UGGGCCUGACCGUGUCCGCCCUGUUCAGCAGGAUCUUCGGCAAGA...,[....(((.(.(.(((((((((((..(((.(((((((((((((......,[-255.0],[0.991]
>ENST00000001146.7|ENSG00000003137.9|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-201|CYP26B1|4556|UTR5:1-28|CDS:29-1567|UTR3:1568-4556|,[UGCUGUUCGAGGGCCUGGACCUGGUGAGCGCCCUGGCCACCCUGG...,[.((((((((..(((((((..(((((((.(((...((((((...))...,"[-825.7, -866.0]","[0.984, 0.974]"
>ENST00000037869.7|ENSG00000035141.8|OTTHUMG00000129668.4|OTTHUMT00000251869.2|FAM136A-201|FAM136A|1810|UTR5:1-79|CDS:80-496|UTR3:497-1810|,[UGGCCGAGCUGCAGCAGCUGCGGGUGCAGGAGGCCGUGGAGUCCA...,[...((((((((.((((((.(((((((((((((((((..(((((((...,"[-195.2, -207.5]","[0.994, 0.982]"
>ENST00000054650.9|ENSG00000041988.16|OTTHUMG00000001440.3|OTTHUMT00000004203.1|THAP3-201|THAP3|1361|UTR5:1-156|CDS:157-876|UTR3:877-1361|,[UGCCCAAGAGCUGUGCUGCUCGGCAGUGCUGCAACCGGUACUCCA...,[.........((((.((((((((((.(((((((....))))))).....,"[-382.3, -424.3, -406.7, -403.5, -400.3, -391.6]","[0.987, 0.962, 0.975, 0.977, 0.979, 0.983]"
>ENST00000054666.11|ENSG00000049245.13|OTTHUMG00000001225.4|OTTHUMT00000003625.2|VAMP3-201|VAMP3|2178|UTR5:1-88|CDS:89-391|UTR3:392-2178|,[UGUCCACCGGGCCUACUGCCGCCACCGGCAGCAACCGGCGGCUGC...,[.(((((((((((.((.((((.(((.((((......)))).))).)...,"[-132.0, -144.7, -140.4, -136.7]","[0.988, 0.972, 0.978, 0.983]"
...,...,...,...,...
>ENST00000695960.1|ENSG00000119844.17|OTTHUMG00000129539.6|-|AFTPH-212|AFTPH|3009|UTR5:1-270|CDS:271-2691|UTR3:2692-3009|,[UGGAGCCCGACAUCAUCAGGAUGUACAGCUCCAGCCCUCCACCCC...,[................(((((((((((((((((.(...(((((.(...,"[-1139.7, -1266.0, -1249.6, -1193.7, -1157.0]","[0.982, 0.963, 0.966, 0.975, 0.98]"
>ENST00000696500.1|ENSG00000054523.20|OTTHUMG00000001817.2|-|KIF1B-215|KIF1B|7839|UTR5:1-388|CDS:389-1255|UTR3:1256-7839|,[UGUCAGGCGCCAGCGUGAAGGUGGCCGUGCGCGUGAGGCCCUUCA...,[..(((((((((((((((..((((..((.(.(((((((((((((.....,"[-394.9, -378.5]","[0.977, 0.983]"
>ENST00000696501.1|ENSG00000054523.20|OTTHUMG00000001817.2|-|KIF1B-216|KIF1B|890|UTR5:1-161|CDS:162-617|UTR3:618-890|,[UGAGCGGGGCCAGCGUGAAGGUGGCCGUGCGGGUGAGGCCCUUCA...,[....(((.(((((.(.....).))))).)))((((((.((.(((....,"[-193.7, -185.4]","[0.98, 0.986]"


In [226]:


agg_LinearDesign_LAMBDA_merged_CDS_gens_unique = LinearDesign_LAMBDA_merged_CDS_gens_unique.groupby('ids').agg(list)

agg_LinearDesign_LAMBDA_merged_CDS_gens_unique['ids'] = agg_LinearDesign_LAMBDA_merged_CDS_gens_unique.index

agg_LinearDesign_LAMBDA_merged_CDS_gens_unique['id'] = agg_LinearDesign_LAMBDA_merged_CDS_gens_unique.apply(lambda x:  [ f"{x['ids'].split('|')[1].split('.')[0]}_NUM{v}" for v in range(1, len(x['mfe'])+1)], axis=1)

LinearDesign_LAMBDA_merged_CDS_gens_unique_new = agg_LinearDesign_LAMBDA_merged_CDS_gens_unique.explode(column=['id', 'mRNA_seq', 'mRNA_structure', 'mfe', 'cai'])
# 
LinearDesign_LAMBDA_merged_CDS_gens_unique_new.reset_index(drop=True, inplace=True)



In [227]:

LinearDesign_LAMBDA_merged_CDS_gens_unique_new.head(1)['ids'].values



array(['>ENST00000000233.10|ENSG00000004059.11|OTTHUMG00000023246.7|OTTHUMT00000059567.3|ARF5-201|ARF5|1032|UTR5:1-88|CDS:89-631|UTR3:632-1032|'],
      dtype=object)

In [228]:

LinearDesign_LAMBDA_merged_CDS_gens_unique_new.to_csv('/home/yul/project/nas_project/bioSPT2/data/interiem/processed/LinearDesign_LAMBDA_merged_CDS_gens_unique_new.csv', 
                                                      index=False,
                                                      sep=',',
                                                      
                                                      )



In [171]:

UTR_genes_human_ts_clean_all.shape

(215, 13)

In [172]:

UTR_genes_human_ts_clean_all.to_csv('/home/yul/project/nas_project/bioSPT2/data/interiem/processed/UTR_genes_human_ts_clean_all.csv',
                                    index=False,
                                    sep=',',
                                    )

In [168]:

mRNAs = pd.DataFrame()

v = 0
for i, cds_row in LinearDesign_LAMBDA_merged_CDS_gens_unique_new.iterrows(): 
    # print(cds_row['id'], cds_row['mRNA_seq'].upper().replace('U', 'T'))
    # print(i, cds_row)
    # if i >3:
    #     break
    
    for j, utr5_row in UTR_genes_human_ts_clean_all.reset_index().iterrows():
        
        # print(utr5_row['Transcript'], utr5_row['UTR5'])
        # if j >3:
        #     break
        
        for k, utr3_row in UTR_genes_human_ts_clean_all[~UTR_genes_human_ts_clean_all['UTR3'].isnull()].reset_index().iterrows():
    
            # print(utr3_row['Transcript'], utr5_row['UTR3'])
            # if k >3:
            #     break
            mRNAs_ = pd.DataFrame(
                {'id': f"CDS_{cds_row['id']}-UTR5_{utr5_row['Transcript']}-UTR3_{utr3_row['Transcript']}",
                 'CDS': cds_row['mRNA_seq'].upper().replace('U', 'T'),
                 'CDS_structure': cds_row['mRNA_structure'],
                 'CDS_mfe': cds_row['mfe'],
                 'CDS_cai': cds_row['cai'],
                 'UTR5': utr5_row['UTR5'],
                 'UTR3': utr3_row['UTR3'],
                 
                 }, index=[v])
            v+=1
            mRNAs = pd.concat([mRNAs, mRNAs_])
            
            if v % 1000 == 0:
                print('processing record: ', v)
            if v % 100000 == 0:
                mRNAs.to_csv('mRNAs_for_finetune.csv', index=False)
                
mRNAs.to_csv('mRNAs_for_finetune.csv', index=False)  



processing record:  1000
processing record:  2000
processing record:  3000
processing record:  4000
processing record:  5000
processing record:  6000
processing record:  7000
processing record:  8000
processing record:  9000
processing record:  10000
processing record:  11000
processing record:  12000
processing record:  13000
processing record:  14000
processing record:  15000
processing record:  16000
processing record:  17000
processing record:  18000
processing record:  19000
processing record:  20000
processing record:  21000
processing record:  22000
processing record:  23000
processing record:  24000
processing record:  25000
processing record:  26000
processing record:  27000
processing record:  28000
processing record:  29000
processing record:  30000
processing record:  31000
processing record:  32000
processing record:  33000
processing record:  34000
processing record:  35000
processing record:  36000
processing record:  37000
processing record:  38000
processing record:  3

KeyboardInterrupt: 

In [ ]:
# /data1/genomics/mRNAs_for_finetune.csv

## RF 配对数据

In [9]:

all_genes_omics

,iBAQ A549_exp,iBAQ HEK293_exp,iBAQ HeLa_exp,iBAQ K562_exp,iBAQ_TransEffect_A549_exp,iBAQ_TransEffect_HEK293_exp,iBAQ_TransEffect_HeLa_exp,iBAQ_TransEffect_K562_exp,iBAQ_TransEffect_A549_rank,iBAQ_TransEffect_HEK293_rank,...,K562_GR_2017_rank,K562_elife_rank,K562_timeLapse_rank,ESC_GR_2017_label,Hela_GSE102113_label,K562_GR_2017_label,K562_elife_label,K562_timeLapse_label,consistent_label_HalfLife,consistent_label_all
ensembl_gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,6.527933,6.796727,5.517671,NaN,5.134571,5.195596,4.181992,NaN,1074.0,2159.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
ENSG00000000419,6.989328,6.970460,6.817138,6.404391,5.275929,5.256304,4.987699,4.315847,756.0,1958.0,...,1926.0,5030.0,1362.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN
ENSG00000000457,4.331123,5.157944,3.871246,NaN,3.588284,4.523821,3.200597,NaN,5498.0,4664.0,...,9413.0,8183.0,NaN,-1.0,NaN,-1.0,-1.0,NaN,NaN,NaN
ENSG00000000460,5.157366,5.762855,4.583522,5.264936,3.871606,4.495252,3.234262,3.624279,4839.0,4755.0,...,5547.0,4374.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN
ENSG00000000938,NaN,4.211548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000285304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3039.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
ENSG00000285330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,160.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
ENSG00000285396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3149.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [ ]:

基本思路：
1）HEK 细胞系一致表达配对
2

In [231]:


all_genes_omics.columns

Index(['iBAQ A549_exp', 'iBAQ HEK293_exp', 'iBAQ HeLa_exp', 'iBAQ K562_exp',
       'iBAQ_TransEffect_A549_exp', 'iBAQ_TransEffect_HEK293_exp',
       'iBAQ_TransEffect_HeLa_exp', 'iBAQ_TransEffect_K562_exp',
       'iBAQ_TransEffect_A549_rank', 'iBAQ_TransEffect_HEK293_rank',
       'iBAQ_TransEffect_HeLa_rank', 'iBAQ_TransEffect_K562_rank',
       'iBAQ_TransEffect_A549_label', 'iBAQ_TransEffect_HEK293_label',
       'iBAQ_TransEffect_HeLa_label', 'iBAQ_TransEffect_K562_label',
       'consistent_label_iBAQ', 'ProteinFamily', 'CopyNumber_Hela_Nagaraj_exp',
       'CopyNumber_Hela_Zecha_exp', 'TransEffect_Hela_Nagaraj_exp',
       'TransEffect_Hela_Zecha_exp', 'TransEffect_Hela_Nagaraj_rank',
       'TransEffect_Hela_Zecha_rank', 'TransEffect_Hela_Nagaraj_label',
       'TransEffect_Hela_Zecha_label', 'consistent_label_CopyNumber',
       'A549-SRR611121-SRR3286543-zhanggong_exp',
       'hela-SRR8443314-SRR8443308_exp', 'HEK293-SRR1795409_10_exp',
       'A549-SRR611121-SRR3286543-zh

In [288]:

# all_genes_omics[
#     [
   
#         'iBAQ_TransEffect_HEK293_exp',
#         'iBAQ_TransEffect_HEK293_rank',
#         'iBAQ_TransEffect_HEK293_label',
#         'HEK293-SRR1795409_10_exp',
#         'HEK293-SRR1795409_10_rank',
#         'HEK293-SRR1795409_10_label',
#         'ESC_GR_2017_exp',
#         'ESC_GR_2017_label',
#         'ESC_GR_2017_rank',
#     ]
# ]

all_genes_omics[
    [
   
        'iBAQ_TransEffect_HEK293_exp',
        'HEK293-SRR1795409_10_exp',
        'ESC_GR_2017_exp',
    ]
].corr(method='spearman')


all_genes_omics[
    [
   
        'iBAQ_TransEffect_HEK293_rank',
        'HEK293-SRR1795409_10_rank',
        'ESC_GR_2017_rank',
    ]
].corr(method='spearman')

all_genes_omics['ESC_GR_2017_exp'].hist()

all_genes_omics['ESC_GR_2017_exp']
# all_genes_omics['ESC_GR_2017_exp']


,iBAQ_TransEffect_HEK293_rank,HEK293-SRR1795409_10_rank,ESC_GR_2017_rank
iBAQ_TransEffect_HEK293_rank,1.000000,0.274423,0.214244
HEK293-SRR1795409_10_rank,0.274423,1.000000,0.178760
ESC_GR_2017_rank,0.214244,0.178760,1.000000


In [239]:

all_genes_omics['iBAQ_TransEffect_HEK293_label'].value_counts()


 0.0    3955
 1.0    1978
-1.0    1978
Name: iBAQ_TransEffect_HEK293_label, dtype: int64

In [10]:

def f(x):
    if x >=2:
        return 1
    elif x<=-2:
        return -1
    else:
        return 0

HEK_ESC_label = all_genes_omics[
    [
   
        'iBAQ_TransEffect_HEK293_label',
        'HEK293-SRR1795409_10_label',
        'ESC_GR_2017_label',
    ]
].sum(axis=1).apply(f)

HEK_ESC_label

ensembl_gene_id
ENSG00000000003    0
ENSG00000000419    1
ENSG00000000457    0
ENSG00000000460    0
ENSG00000000938    0
                  ..
ENSG00000285304    0
ENSG00000285330    0
ENSG00000285396    0
ENSG00000285441    0
ENSG00000288825    0
Length: 19868, dtype: int64

In [11]:

HEK_ESC_data = human_ts.set_index('Esemble').reindex(HEK_ESC_label[HEK_ESC_label!=0].index)
HEK_ESC_data.head()

,Transcript,gene_id,name,seq,transcript_id,gene_id_raw,gene_name,UTR5_regs,CDS_regs,UTR3_regs,UTR5,CDS,UTR3
ensembl_gene_id,,,,,,,,,,,,,
ENSG00000000419,ENST00000371588,ENSG00000000419.14,>ENST00000371588.10|ENSG00000000419.14|OTTHUMG...,AGTTCCGCCATGGCCTCCTTGGAAGTCAGTCGTAGTCCTCGCAGGT...,ENST00000371588.10,ENST00000371588.10,DPM1,UTR5:1-9,CDS:10-792,UTR3:793-1054,AGTTCCGCC,ATGGCCTCCTTGGAAGTCAGTCGTAGTCCTCGCAGGTCTCGGCGGG...,AAGAAAGATACTCATTTATAGTTACGTTCATTTCAGGTTAAACATG...
ENSG00000001497,ENST00000374811,ENSG00000001497.18,>ENST00000374811.8|ENSG00000001497.18|OTTHUMG0...,GGAGTGAGGGTAGCACGCTGAGCTGAAGGCTGTGCGGAGCGGCGCG...,ENST00000374811.8,ENST00000374811.8,LAS1L,UTR5:1-72,CDS:73-2277,UTR3:2278-2470,GGAGTGAGGGTAGCACGCTGAGCTGAAGGCTGTGCGGAGCGGCGCG...,ATGTCGTGGGAATCCGGGGCCGGGCCAGGTCTAGGTTCCCAGGGGA...,TGGCCATCCCTGGTGCAAGTGTTCATCCAGCCGTGCCAGGGCAACA...
ENSG00000002549,ENST00000606142,ENSG00000002549.13,>ENST00000606142.5|ENSG00000002549.13|OTTHUMG0...,GCGAGTAGTCGTCCGACGTCTGGCCGTGAGACGTTTCGGGAGCCGG...,ENST00000606142.5,ENST00000606142.5,LAP3,UTR5:1-64,CDS:65-1531,UTR3:1532-1876,GCGAGTAGTCGTCCGACGTCTGGCCGTGAGACGTTTCGGGAGCCGG...,ATGACGAAGGGCCTTGTTTTAGGAATCTATTCCAAAGAAAAAGAAG...,TTCAGATACTCAAAAATGTCTTCACTCTGTCTTAAATTGGACAGTT...
ENSG00000002834,ENST00000318008,ENSG00000002834.18,>ENST00000318008.11|ENSG00000002834.18|OTTHUMG...,GCCAGTTCCCCAGCTCCAGCCGCCGTCGCTGCTGCCTGTGTAGTTG...,ENST00000318008.11,ENST00000318008.11,LASP1,UTR5:1-132,CDS:133-918,UTR3:919-3910,GCCAGTTCCCCAGCTCCAGCCGCCGTCGCTGCTGCCTGTGTAGTTG...,ATGAACCCCAACTGCGCCCGGTGCGGCAAGATCGTGTATCCCACGG...,ACCCGCAGCGCCCCCATCTGTCTTCAGCACATTCCACGGCATCGCA...
ENSG00000004478,ENST00000001008,ENSG00000004478.8,>ENST00000001008.6|ENSG00000004478.8|OTTHUMG00...,CCTACCCCAGCTCTCGCGCCGCGTGCAGAGGTGCTCAAGCCTCCTC...,ENST00000001008.6,ENST00000001008.6,FKBP4,UTR5:1-170,CDS:171-1550,UTR3:1551-3715,CCTACCCCAGCTCTCGCGCCGCGTGCAGAGGTGCTCAAGCCTCCTC...,ATGACAGCCGAGGAGATGAAGGCGACCGAGAGCGGGGCGCAGTCGG...,CCCCTCTCCACCAGCCCTACTCCTGCGGCTGCCTGCCCCCCAGTCT...


In [27]:

# row["CDS"]

HEK_ESC_data


,Transcript,gene_id,name,seq,transcript_id,gene_id_raw,gene_name,UTR5_regs,CDS_regs,UTR3_regs,UTR5,CDS,UTR3
ensembl_gene_id,,,,,,,,,,,,,
ENSG00000000419,ENST00000371588,ENSG00000000419.14,>ENST00000371588.10|ENSG00000000419.14|OTTHUMG...,AGTTCCGCCATGGCCTCCTTGGAAGTCAGTCGTAGTCCTCGCAGGT...,ENST00000371588.10,ENST00000371588.10,DPM1,UTR5:1-9,CDS:10-792,UTR3:793-1054,AGTTCCGCC,ATGGCCTCCTTGGAAGTCAGTCGTAGTCCTCGCAGGTCTCGGCGGG...,AAGAAAGATACTCATTTATAGTTACGTTCATTTCAGGTTAAACATG...
ENSG00000001497,ENST00000374811,ENSG00000001497.18,>ENST00000374811.8|ENSG00000001497.18|OTTHUMG0...,GGAGTGAGGGTAGCACGCTGAGCTGAAGGCTGTGCGGAGCGGCGCG...,ENST00000374811.8,ENST00000374811.8,LAS1L,UTR5:1-72,CDS:73-2277,UTR3:2278-2470,GGAGTGAGGGTAGCACGCTGAGCTGAAGGCTGTGCGGAGCGGCGCG...,ATGTCGTGGGAATCCGGGGCCGGGCCAGGTCTAGGTTCCCAGGGGA...,TGGCCATCCCTGGTGCAAGTGTTCATCCAGCCGTGCCAGGGCAACA...
ENSG00000002549,ENST00000606142,ENSG00000002549.13,>ENST00000606142.5|ENSG00000002549.13|OTTHUMG0...,GCGAGTAGTCGTCCGACGTCTGGCCGTGAGACGTTTCGGGAGCCGG...,ENST00000606142.5,ENST00000606142.5,LAP3,UTR5:1-64,CDS:65-1531,UTR3:1532-1876,GCGAGTAGTCGTCCGACGTCTGGCCGTGAGACGTTTCGGGAGCCGG...,ATGACGAAGGGCCTTGTTTTAGGAATCTATTCCAAAGAAAAAGAAG...,TTCAGATACTCAAAAATGTCTTCACTCTGTCTTAAATTGGACAGTT...
ENSG00000002834,ENST00000318008,ENSG00000002834.18,>ENST00000318008.11|ENSG00000002834.18|OTTHUMG...,GCCAGTTCCCCAGCTCCAGCCGCCGTCGCTGCTGCCTGTGTAGTTG...,ENST00000318008.11,ENST00000318008.11,LASP1,UTR5:1-132,CDS:133-918,UTR3:919-3910,GCCAGTTCCCCAGCTCCAGCCGCCGTCGCTGCTGCCTGTGTAGTTG...,ATGAACCCCAACTGCGCCCGGTGCGGCAAGATCGTGTATCCCACGG...,ACCCGCAGCGCCCCCATCTGTCTTCAGCACATTCCACGGCATCGCA...
ENSG00000004478,ENST00000001008,ENSG00000004478.8,>ENST00000001008.6|ENSG00000004478.8|OTTHUMG00...,CCTACCCCAGCTCTCGCGCCGCGTGCAGAGGTGCTCAAGCCTCCTC...,ENST00000001008.6,ENST00000001008.6,FKBP4,UTR5:1-170,CDS:171-1550,UTR3:1551-3715,CCTACCCCAGCTCTCGCGCCGCGTGCAGAGGTGCTCAAGCCTCCTC...,ATGACAGCCGAGGAGATGAAGGCGACCGAGAGCGGGGCGCAGTCGG...,CCCCTCTCCACCAGCCCTACTCCTGCGGCTGCCTGCCCCCCAGTCT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000260455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000261115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000261326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [255]:

with open('/home/yul/project/nas_project/bioSPT2/data/interiem/processed/HEK_ESC_data.fasta', 'w') as f:
    for index, row in HEK_ESC_data.iterrows():
        f.write(f'>{index}\n{row["CDS"]}\n')

In [32]:
# 提取聚类簇

cdhit_pd = pd.DataFrame()
with open('/data1/genomics/yul/bioSPT2/data/interiem/processed/HEK_ESC_data_db_0.8.clstr', 'r') as f:
    
    lines=f.readlines()
    
    group_ids = []
    groups = []
    for line in lines:
        if line.startswith('>'):
            group_id = line.strip()
            group_ids.append(group_id)
            group_items = []
            groups.append(group_items)
        else:
            group_item = line.strip().split('>')[-1].strip().split('...')[0]
            group_items.append(group_item)
            
    groups.append(group_items)
    

cdhit_pd = pd.DataFrame(
    {
        'group_id': group_ids,
        'group_items': groups[1:]
    }
)

rf_cdhit_pd = cdhit_pd[cdhit_pd['group_items'].apply(len) > 1]

rf_cdhit_pd['group_item_labels'] = rf_cdhit_pd['group_items'].apply(lambda vars: [HEK_ESC_label.reindex([v])[0] for v in vars])

rf_cdhit_pd['group_item_ESC_values'] = rf_cdhit_pd['group_items'].apply(lambda vars: [all_genes_omics.reindex([v])['ESC_GR_2017_exp'][0] for v in vars]) 

rf_cdhit_pd['group_item_iBAQ_values'] = rf_cdhit_pd['group_items'].apply(lambda vars: [all_genes_omics.reindex([v])['iBAQ_TransEffect_HEK293_exp'][0] for v in vars]) 

rf_cdhit_pd['group_item_TE_values'] = rf_cdhit_pd['group_items'].apply(lambda vars: [all_genes_omics.reindex([v])['HEK293-SRR1795409_10_exp'][0] for v in vars])

rf_cdhit_pd.head()



/tmp/ipykernel_17137/1899396817.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rf_cdhit_pd['group_item_labels'] = rf_cdhit_pd['group_items'].apply(lambda vars: [HEK_ESC_label.reindex([v])[0] for v in vars])
/tmp/ipykernel_17137/1899396817.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rf_cdhit_pd['group_item_ESC_values'] = rf_cdhit_pd['group_items'].apply(lambda vars: [all_genes_omics.reindex([v])['ESC_GR_2017_exp'][0] for v in vars])
/tmp/ipykernel_17137/1899396817.py:36: SettingWithCopyWarning

,group_id,group_items,group_item_labels,group_item_ESC_values,group_item_iBAQ_values,group_item_TE_values
208,>Cluster 208,"[ENSG00000160336, ENSG00000213799, ENSG0000019...","[-1, -1, -1]","[nan, 1.2009223333333334, 2.2410646666666665]","[3.026569772605642, 3.981287824628937, nan]","[0.080205, 0.256335, 0.063765]"
243,>Cluster 243,"[ENSG00000072110, ENSG00000130402]","[1, 1]","[nan, nan]","[5.638357939299184, 6.182171418362166]","[1.123115, 2.271605]"
462,>Cluster 462,"[ENSG00000178935, ENSG00000269343]","[-1, -1]","[1.7730736666666669, 1.8657866666666667]","[3.567627576953936, nan]","[0.06825, 0.012015]"
473,>Cluster 473,"[ENSG00000178229, ENSG00000083844]","[-1, -1]","[1.0260586, 1.2323676666666668]","[3.011159736350703, nan]","[0.468535, 0.075545]"
586,>Cluster 586,"[ENSG00000182141, ENSG00000237440]","[-1, -1]","[2.041819333333333, 1.306525]","[2.775471759967293, nan]","[0.0962, 0.06184]"


In [33]:

pairs = []
for index, row in rf_cdhit_pd.iterrows():
    
    logb = 1.3
    lb, ub  = 1/logb, logb    
    
    print(index)
    for i in range(len(row['group_items'])-1):
        for j in range(i+1, len(row['group_items'])):
            pair = []
            pair.extend([row['group_items'][i], row['group_items'][j]])
            if row['group_item_ESC_values'][i] and row['group_item_ESC_values'][j]:
                if row['group_item_ESC_values'][i]/row['group_item_ESC_values'][j] >=ub:
                    pair.append(1)
                    
                elif row['group_item_ESC_values'][i]/row['group_item_ESC_values'][j] <= lb:
                    pair.append(-1)
                    
                else:
                    pair.append(0)
            else:
                pair.append(None)
        
            if row['group_item_TE_values'][i] and row['group_item_TE_values'][j]:
                if row['group_item_TE_values'][i]/row['group_item_TE_values'][j] >=ub:
                    pair.append(1)
                    
                elif row['group_item_TE_values'][i]/row['group_item_TE_values'][j] <= lb:
                    pair.append(-1)
                    
                else:
                    pair.append(0)
            else:
                pair.append(None)
                    
            if row['group_item_iBAQ_values'][i] and row['group_item_iBAQ_values'][j]:
                if row['group_item_iBAQ_values'][i]/row['group_item_iBAQ_values'][j] >=ub:
                    pair.append(1)
                    
                elif row['group_item_iBAQ_values'][i]/row['group_item_iBAQ_values'][j] <= lb:
                    pair.append(-1)
                    
                else:
                    pair.append(0)
            else:
                pair.append(None)
                
            print(pair)
            pairs.append(pair)
            

208
['ENSG00000160336', 'ENSG00000213799', 0, -1, -1]
['ENSG00000160336', 'ENSG00000196417', 0, 0, 0]
['ENSG00000213799', 'ENSG00000196417', -1, 1, 0]
243
['ENSG00000072110', 'ENSG00000130402', 0, -1, 0]
462
['ENSG00000178935', 'ENSG00000269343', 0, 1, 0]
473
['ENSG00000178229', 'ENSG00000083844', 0, 1, 0]
586
['ENSG00000182141', 'ENSG00000237440', 1, 1, 0]
602
['ENSG00000083812', 'ENSG00000249471', 0, 1, 0]
620
['ENSG00000187626', 'ENSG00000189298', -1, 1, 0]
757
['ENSG00000130741', 'ENSG00000180574', 1, 0, 0]
813
['ENSG00000104833', 'ENSG00000137267', -1, 1, 0]
['ENSG00000104833', 'ENSG00000137285', -1, 0, 0]
['ENSG00000104833', 'ENSG00000188229', 0, 1, 0]
['ENSG00000104833', 'ENSG00000196230', -1, 0, 0]
['ENSG00000104833', 'ENSG00000258947', 0, 1, 0]
['ENSG00000137267', 'ENSG00000137285', -1, 0, 1]
['ENSG00000137267', 'ENSG00000188229', 1, 0, 0]
['ENSG00000137267', 'ENSG00000196230', -1, 0, 0]
['ENSG00000137267', 'ENSG00000258947', 0, 1, 0]
['ENSG00000137285', 'ENSG00000188229', 1, 

In [34]:

RF_exp_pairs = pd.DataFrame(pairs, columns=['gene1', 'gene2', 'ESC', 'TE', 'iBAQ'])


def label(x):
    
    cols = ['ESC', 'TE', 'iBAQ']
    
    if all([x[c] >=0 for c in cols]) or all([x[c] <=0 for c in cols]):
        if sum([x[c] for c in cols]) > 0:
            return 1
        elif sum([x[c] for c in cols]) < 0:
            return -1
        else:
            return 0
    return 0
    
RF_exp_pairs['pair_label'] = RF_exp_pairs.apply(label, axis=1)

RF_exp_pairs



,gene1,gene2,ESC,TE,iBAQ,pair_label
0,ENSG00000160336,ENSG00000213799,0,-1,-1,-1
1,ENSG00000160336,ENSG00000196417,0,0,0,0
2,ENSG00000213799,ENSG00000196417,-1,1,0,0
3,ENSG00000072110,ENSG00000130402,0,-1,0,-1
4,ENSG00000178935,ENSG00000269343,0,1,0,1
...,...,...,...,...,...,...
59,ENSG00000270276,ENSG00000277157,0,0,0,0
60,ENSG00000270276,ENSG00000278637,0,0,0,0
61,ENSG00000276180,ENSG00000277157,0,-1,0,-1
62,ENSG00000276180,ENSG00000278637,0,-1,0,-1


In [35]:

RF_exp_pairs.to_csv('/home/yul/project/nas_project/bioSPT2/data/interiem/processed/HEK_ESC_data_db_0.8_pairs_expression.tsv', sep='\t')


#### utr5

In [26]:
# 读取 湿实验数据

experiment_exp = pd.read_excel(
    
    '/home/yul/project/nas_project/bioSPT2/data/interiem/processed/UTR5_experiment_validation_results_summary.xlsx'
)

experiment_exp.head()


,Unnamed: 0,experiment_id,experiment_batch,experiment_enzyme,enzyme_seq,sequence_adapter,cap_seq,seq_name,seq,template_seq,template_name,gene,is_template,UTR5,CDS,CDS_name,UTR3,expression
0,47,14,v1,BamHI,GGATCC,TAATACGACTCACTATA,GGG,template@Neocura_5UTR,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,Neocura_5UTR,NaN,True,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,degron-No.15,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,1.650000
1,64,14,v1,BamHI,GGATCC,TAATACGACTCACTATA,GGG,template@Neocura_5UTR,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,Neocura_5UTR,NaN,True,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGTGTCCAAGGGGGAGGAGCTGTTCACCGGCGTGGTGCCCATCC...,degron-No.22,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,1.500000
2,86,No.39,v2,EcoRI,GAATTC,TAATACGACTCACTATA,GGA,69_1_ENSG00000163220,AAGCGGTATCGGTGTGCGAGTCTTGCGGTTGTCTCTCTTCGCTCTT...,AAACACTCTGTGTGGCTCCTCGGCTTTGACAGAGTGCAAGACG,ENSG00000163220,ENSG00000163220,False,GGAAAGCGGTATCGGTGTGCGAGTCTTGCGGTTGTCTCTCTTCGCT...,AGCAGAGTGCAGCGGGTGACAGACAGCCTGGACACACTGCTGGAGA...,AG2-25,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,1.433288
3,88,No.41,v2,EcoRI,GAATTC,TAATACGACTCACTATA,GGA,171_1_ENSG00000163220,ACTTCTCGAGTTTTGGTGTTTTGTCGGTTCCGCTGTTGCGTTGTGG...,AAACACTCTGTGTGGCTCCTCGGCTTTGACAGAGTGCAAGACG,ENSG00000163220,ENSG00000163220,False,GGAACTTCTCGAGTTTTGGTGTTTTGTCGGTTCCGCTGTTGCGTTG...,AGCAGAGTGCAGCGGGTGACAGACAGCCTGGACACACTGCTGGAGA...,AG2-25,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,1.404425
4,53,No.38,v2,EcoRI,GAATTC,TAATACGACTCACTATA,GGA,65_1_ENSG00000163220,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,AAACACTCTGTGTGGCTCCTCGGCTTTGACAGAGTGCAAGACG,ENSG00000163220,ENSG00000163220,False,GGAAAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTT...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,degron-No.5,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,1.389417


In [27]:

experiment_exp_agg = experiment_exp.groupby(by=['UTR5', 'CDS', 'UTR3']).agg(list)
agged_cols = list(experiment_exp_agg.columns)
experiment_exp_agg['gene_id'] = [f'gene_{g}' for g in range(experiment_exp_agg.shape[0])]

experiment_exp_agg_exploded = experiment_exp_agg.explode(agged_cols)

experiment_exp_agg_exploded = experiment_exp_agg_exploded.reset_index()

In [31]:


pairs = []
cutoff  = 0.3
for i_index, i_row in experiment_exp_agg_exploded.iterrows():
    for j_index, j_row in experiment_exp_agg_exploded.iterrows():
    
        if i_row['expression'] and j_row['expression'] and \
            i_row['gene_id'] != j_row['gene_id'] and \
                i_row['CDS'] == j_row['CDS']:
            pair = [i_row['gene_id'], j_row['gene_id']]
            if i_row['expression'] - j_row['expression'] >= cutoff:
                pair.append(1)
                
            elif i_row['expression'] - j_row['expression'] <= -cutoff:
                pair.append(-1)
                
            else:
                pair.append(0)
                
            pairs.append(pair)

In [ ]:
# 配对
pairs = []
cutoff  = 0 # 0.3
for i_index, i_row in experiment_exp_agg_exploded.iterrows():
    for j_index, j_row in experiment_exp_agg_exploded.iterrows():
    
        if i_row['expression'] and j_row['expression'] and \
            i_row['gene_id'] != j_row['gene_id'] and \
                i_row['CDS'] == j_row['CDS']:
            pair = [i_row['gene_id'], j_row['gene_id']]
            if i_row['expression']/j_row['expression'] >= (1+cutoff):
                pair.append(1)
                
            elif i_row['expression']/j_row['expression'] <= 1/(1+cutoff):
                pair.append(-1)
                
            else:
                pair.append(0)
                
            pair.append(i_row['expression']/j_row['expression'])
            
            pairs.append(pair)
            
            
            

In [29]:

# experiment_exp_agg.reset_index().set_index('gene_id').reindex(['gene_0'])['UTR5'].values[0]
len(pairs)

8780

In [32]:

experiment_pair_exp = pd.DataFrame(pairs, columns=['pair_A', 'pair_B', 'pair_label', 'pair_diff'])

experiment_pair_exp['pair_A_UTR5'] = experiment_pair_exp['pair_A'].apply(lambda x: experiment_exp_agg.reset_index().set_index('gene_id').reindex([x])['UTR5'].values[0])
experiment_pair_exp['pair_A_UTR3'] = experiment_pair_exp['pair_A'].apply(lambda x: experiment_exp_agg.reset_index().set_index('gene_id').reindex([x])['UTR3'].values[0])
experiment_pair_exp['pair_A_CDS'] = experiment_pair_exp['pair_A'].apply(lambda x: experiment_exp_agg.reset_index().set_index('gene_id').reindex([x])['CDS'].values[0])

experiment_pair_exp['pair_B_UTR5'] = experiment_pair_exp['pair_B'].apply(lambda x: experiment_exp_agg.reset_index().set_index('gene_id').reindex([x])['UTR5'].values[0])
experiment_pair_exp['pair_B_UTR3'] = experiment_pair_exp['pair_B'].apply(lambda x: experiment_exp_agg.reset_index().set_index('gene_id').reindex([x])['UTR3'].values[0])
experiment_pair_exp['pair_B_CDS'] = experiment_pair_exp['pair_B'].apply(lambda x: experiment_exp_agg.reset_index().set_index('gene_id').reindex([x])['CDS'].values[0])


experiment_pair_exp



,pair_A,pair_B,pair_label,pair_A_UTR5,pair_A_UTR3,pair_A_CDS,pair_B_UTR5,pair_B_UTR3,pair_B_CDS
0,gene_0,gene_52,1,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GGGAAATAAGAGAGAAAAGAAGAGTAAGAAGAAATATAAGACCCCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...
1,gene_0,gene_56,1,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...
2,gene_0,gene_60,1,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GGGAATCAACACAGACACCAGACCACCAGCACCAGAACACACAGAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...
3,gene_0,gene_64,1,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GGGAATCACCACCAGAACACACACACAGAAACACACAGACACCAGA...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...
4,gene_0,gene_68,1,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GGGAATTCGTCGACTCAGGAACTATTCTTCTGGTCCCCACAGACTC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...
...,...,...,...,...,...,...,...,...,...
8775,gene_115,gene_95,0,GGGTCTCAACACAACATATACAAAACAAACGAATCTCAAGCAATCA...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGTCTAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GGGATTCCCCCACCACCAGAGCACCAGTCCCGCAGTCGGCGTCCAG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGTCTAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...
8776,gene_115,gene_99,0,GGGTCTCAACACAACATATACAAAACAAACGAATCTCAAGCAATCA...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGTCTAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GGGATTCCCCGCCTCCTCCTGTCCCGCAGTCGGCGTCCAGCGGCTC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGTCTAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...
8777,gene_115,gene_103,1,GGGTCTCAACACAACATATACAAAACAAACGAATCTCAAGCAATCA...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGTCTAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GGGGCCACCAAGTTGTTACATGAAATCTGCAGTTTCATAATTTCCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGTCTAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...
8778,gene_115,gene_107,1,GGGTCTCAACACAACATATACAAAACAAACGAATCTCAAGCAATCA...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGTCTAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GGGGCCAGCAGAAGTTGTTACAAATCTGCAGTTTCATTTCCGTGGG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGTCTAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...


In [33]:

experiment_pair_exp.to_csv(f'data/interiem/processed/experiment_UTR5_pair_exp_cutoff{cutoff}.tsv', sep='\t')


#### UTR3

In [34]:

experiment_UTR3_exp = pd.read_excel(
    
    '/home/yul/project/nas_project/bioSPT2/data/interiem/processed/Neocura_3UTR_validation_results_062024.xlsx', header=1
)

experiment_UTR3_exp.head()

,No.,ID-Wet,Name-Wet,ID-Dry,Note-Dry,3-nt,5'UTR,CDS,3'UTR,5‘UTR-CDS,...,NA4.2,Test1.2,Test2.2,Test3.2,Test4.2,Ref1.2,Ref2.2,Ref3.2,Ref4.2,Ref seq (Name-Wet).2
0,1,3'UTR-1,pNC-eGFPdegron5-3'UTR-1,NaN,BNT_1,GGG,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,CTGGTACTGGAATAAGCACGAATTAATGCTAGCTGCCCCTAAAGCT...,ggatcc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,3'UTR-2,pNC-eGFPdegron5-3'UTR-2,NaN,BNT_2,GGG,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,CTGGTACTGCCCCTGCTGCACCGCCATGCTAGCTGGCTGTCCCGTC...,ggatcc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3'UTR-3,pNC-eGFPdegron5-3'UTR-3,NaN,BNT_3,GGG,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,CTGGGCAGTAAAGGCACGCAATGCTTGCTGCTGCCCCTCCTGTGCG...,ggatcc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,3'UTR-4,pNC-eGFPdegron5-3'UTR-4,NaN,BNT_4,GGG,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,CTTCCGCCGTACTGCATCCTGGCACGCAATGCTAGCTGCCCCTTTC...,ggatcc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,3'UTR-5,pNC-eGFPdegron5-3'UTR-5,NaN,Moderna_1,GGG,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GCTTCGTGTCTTCTGTGGCCTAGCTTCCCTGCTCTTGGGCCTCCCC...,ggatcc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:

agged_cols = experiment_UTR3_exp.columns



Index(['No.', 'ID-Wet', 'Name-Wet', 'ID-Dry', 'Note-Dry', '3-nt', '5'UTR',
       'CDS', '3'UTR', '5‘UTR-CDS', 'CDS-3‘UTR', '3‘UTR-polyA', 'polyA',
       'Linearization Enzyme', 'Comeplete Seq', 'Cell-line', 'Test Date',
       'NA1', 'NA2', 'NA3', 'NA4', 'Test1', 'Test2', 'Test3', 'Test4', 'Ref1',
       'Ref2', 'Ref3', 'Ref4', 'Ref seq (Name-Wet)', 'Cell-line.1',
       'Test Date.1', 'NA1.1', 'NA2.1', 'NA3.1', 'NA4.1', 'Test1.1', 'Test2.1',
       'Test3.1', 'Test4.1', 'Ref1.1', 'Ref2.1', 'Ref3.1', 'Ref4.1',
       'Ref seq (Name-Wet).1', 'Cell-line.2', 'Test Date.2', 'NA1.2', 'NA2.2',
       'NA3.2', 'NA4.2', 'Test1.2', 'Test2.2', 'Test3.2', 'Test4.2', 'Ref1.2',
       'Ref2.2', 'Ref3.2', 'Ref4.2', 'Ref seq (Name-Wet).2', 'mean_exp'],
      dtype='object')

In [16]:

experiment_UTR3_exp[
    ['NA1', 'NA2', 'NA3', 'NA4',
     'Test1', 'Test2', 'Test3', 'Test4',
     'NA1.1', 'NA2.1', 'NA3.1', 'NA4.1', 'Test1.1', 'Test2.1',
       'Test3.1', 'Test4.1',
     'NA1.2', 'NA2.2', 'NA3.2', 'NA4.2',
     'Test1.2', 'Test2.2', 'Test3.2', 'Test4.2',
     ]
                    ].head(20)




,NA1,NA2,NA3,NA4,Test1,Test2,Test3,Test4,NA1.1,NA2.1,...,Test3.1,Test4.1,NA1.2,NA2.2,NA3.2,NA4.2,Test1.2,Test2.2,Test3.2,Test4.2
0,8.31,8.31,8.31,8.31,3202.0,3017.0,3540.0,3453.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.31,8.31,8.31,8.31,7390.0,7050.0,8280.0,8091.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8.31,8.31,8.31,8.31,10709.0,9974.0,10965.0,9778.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.31,8.31,8.31,8.31,8801.0,7830.0,9147.0,8806.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8.31,8.31,8.31,8.31,12652.0,12386.0,13643.0,12911.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,8.31,8.31,8.31,8.31,15304.0,15034.0,14655.0,15721.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,8.31,8.31,8.31,8.31,21254.0,19505.0,21539.0,20411.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8.31,8.31,8.31,8.31,15648.0,15528.0,17036.0,16464.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8.31,8.31,8.31,8.31,19017.0,18885.0,19263.0,18886.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,8.31,8.31,8.31,8.31,9857.0,9756.0,11010.0,11626.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:

experiment_UTR3_exp['mean_exp'] =  experiment_UTR3_exp[
    [
     'Test1', 'Test2', 'Test3', 'Test4',
      'Test1.1', 'Test2.1','Test3.1', 'Test4.1',
     'Test1.2', 'Test2.2', 'Test3.2', 'Test4.2',
     ]
                    ].mean(axis=1)

In [36]:

experiment_UTR3_exp_agg = experiment_UTR3_exp.groupby(by=['3-nt', "5'UTR", 'CDS', "3'UTR", '5‘UTR-CDS', 'CDS-3‘UTR']).agg(list)

agged_cols = list(experiment_UTR3_exp_agg.columns)

experiment_UTR3_exp_agg['gene_id'] = [f'gene_{g}' for g in range(experiment_UTR3_exp_agg.shape[0])]

experiment_UTR3_exp_agg_exploded = experiment_UTR3_exp_agg.explode(agged_cols)

experiment_UTR3_exp_agg_exploded = experiment_UTR3_exp_agg_exploded.reset_index()

experiment_UTR3_exp_agg_exploded.head()

,3-nt,5'UTR,CDS,3'UTR,5‘UTR-CDS,CDS-3‘UTR,No.,ID-Wet,Name-Wet,ID-Dry,...,Test2.2,Test3.2,Test4.2,Ref1.2,Ref2.2,Ref3.2,Ref4.2,Ref seq (Name-Wet).2,mean_exp,gene_id
0,AGG,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...,ACTAATACTGGTCTGCGGCAGCTAAATAAGATCAATAAATTAATTC...,-,-,151,3'UTR-58,018-3‘UTR-58,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1288.2,gene_0
1,AGG,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...,ACTGTACCATTTACCACTGTGACTAAATAAAGGGTGTGTCTGTTGT...,-,-,157,3'UTR-64,018-3‘UTR-64,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1209.2,gene_1
2,AGG,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...,CCCCCAATATCTGCTCCTCTGGTTTGGCTTCCCTCAGAGCCTGGGT...,-,-,124,3'UTR-31,018-3‘UTR-31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3076.6,gene_2
3,AGG,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...,CCCCGTCTGCTCACCTGCAAAATAAAAGGCTTTTCGCCCCTTGTCC...,-,-,126,3'UTR-33,018-3‘UTR-33,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2961.4,gene_3
4,AGG,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...,CCCCGTCTGCTCCTCTAGGCTGGCCCCTTGTCCACCCCTCTCCACA...,-,-,138,3'UTR-45,018-3‘UTR-45,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2973.0,gene_4


In [38]:


pairs = []
for i_index, i_row in experiment_UTR3_exp_agg_exploded.iterrows():
    for j_index, j_row in experiment_UTR3_exp_agg_exploded.iterrows():
    
        if i_row['mean_exp'] and j_row['mean_exp'] and \
            i_row['gene_id'] != j_row['gene_id'] and \
                i_row['CDS'] == j_row['CDS']:
      
            pair = [i_row['gene_id'], j_row['gene_id']]
            if i_row['mean_exp']/j_row['mean_exp'] >= (1+cutoff):
                pair.append(1)
                
            elif i_row['mean_exp']/j_row['mean_exp'] <= 1/(1+cutoff):
                pair.append(-1)
                
            else:
                pair.append(0)
                
            pair.append(i_row['mean_exp']/j_row['mean_exp'])
                
            pairs.append(pair)
            
            

In [44]:
len(pairs)

6846

In [39]:

experiment_UTR3_pair_exp = pd.DataFrame(pairs, columns=['pair_A', 'pair_B', 'pair_label', 'pair_diff'])

experiment_UTR3_pair_exp['pair_A_UTR5'] = experiment_UTR3_pair_exp['pair_A'].apply(lambda x: experiment_UTR3_exp_agg.reset_index().set_index('gene_id').reindex([x])["5'UTR"].values[0])
experiment_UTR3_pair_exp['pair_A_UTR3'] = experiment_UTR3_pair_exp['pair_A'].apply(lambda x: experiment_UTR3_exp_agg.reset_index().set_index('gene_id').reindex([x])["3'UTR"].values[0])
experiment_UTR3_pair_exp['pair_A_CDS'] = experiment_UTR3_pair_exp['pair_A'].apply(lambda x: experiment_UTR3_exp_agg.reset_index().set_index('gene_id').reindex([x])['CDS'].values[0])

experiment_UTR3_pair_exp['pair_B_UTR5'] = experiment_UTR3_pair_exp['pair_B'].apply(lambda x: experiment_UTR3_exp_agg.reset_index().set_index('gene_id').reindex([x])["5'UTR"].values[0])
experiment_UTR3_pair_exp['pair_B_UTR3'] = experiment_UTR3_pair_exp['pair_B'].apply(lambda x: experiment_UTR3_exp_agg.reset_index().set_index('gene_id').reindex([x])["3'UTR"].values[0])
experiment_UTR3_pair_exp['pair_B_CDS'] = experiment_UTR3_pair_exp['pair_B'].apply(lambda x: experiment_UTR3_exp_agg.reset_index().set_index('gene_id').reindex([x])['CDS'].values[0])


experiment_UTR3_pair_exp

,pair_A,pair_B,pair_label,pair_A_UTR5,pair_A_UTR3,pair_A_CDS,pair_B_UTR5,pair_B_UTR3,pair_B_CDS
0,gene_0,gene_1,0,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ACTAATACTGGTCTGCGGCAGCTAAATAAGATCAATAAATTAATTC...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ACTGTACCATTTACCACTGTGACTAAATAAAGGGTGTGTCTGTTGT...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...
1,gene_0,gene_2,-1,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ACTAATACTGGTCTGCGGCAGCTAAATAAGATCAATAAATTAATTC...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,CCCCCAATATCTGCTCCTCTGGTTTGGCTTCCCTCAGAGCCTGGGT...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...
2,gene_0,gene_3,-1,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ACTAATACTGGTCTGCGGCAGCTAAATAAGATCAATAAATTAATTC...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,CCCCGTCTGCTCACCTGCAAAATAAAAGGCTTTTCGCCCCTTGTCC...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...
3,gene_0,gene_4,-1,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ACTAATACTGGTCTGCGGCAGCTAAATAAGATCAATAAATTAATTC...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,CCCCGTCTGCTCCTCTAGGCTGGCCCCTTGTCCACCCCTCTCCACA...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...
4,gene_0,gene_5,-1,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ACTAATACTGGTCTGCGGCAGCTAAATAAGATCAATAAATTAATTC...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,CCCCTTCTGTTCTGCTCCTCTAATAAAGGCTCTGATTTTCCCCCAT...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...
...,...,...,...,...,...,...,...,...,...
6841,gene_156,gene_151,-1,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,TTGTGTATGCGTTAATAAAAAGAAGGAACTCGTATTGTGTATGCGT...,ATGGTGTCCAAGGGGGAGGAGCTGTTCACCGGCGTGGTGCCCATCC...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,GCTTCGTGTCTTCTGTGGCCTAGCTTCCCTGCTCTTGGGCCTCCCC...,ATGGTGTCCAAGGGGGAGGAGCTGTTCACCGGCGTGGTGCCCATCC...
6842,gene_156,gene_152,-1,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,TTGTGTATGCGTTAATAAAAAGAAGGAACTCGTATTGTGTATGCGT...,ATGGTGTCCAAGGGGGAGGAGCTGTTCACCGGCGTGGTGCCCATCC...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,TTGTAACTCCTGCGTTAAATAAAAAGAAGGAACTCGTAAAAACTCT...,ATGGTGTCCAAGGGGGAGGAGCTGTTCACCGGCGTGGTGCCCATCC...
6843,gene_156,gene_153,1,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,TTGTGTATGCGTTAATAAAAAGAAGGAACTCGTATTGTGTATGCGT...,ATGGTGTCCAAGGGGGAGGAGCTGTTCACCGGCGTGGTGCCCATCC...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,TTGTGTATGCGTTAATAAAAAGAAGGAACTCGTAAAAACTAATCTT...,ATGGTGTCCAAGGGGGAGGAGCTGTTCACCGGCGTGGTGCCCATCC...
6844,gene_156,gene_154,-1,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,TTGTGTATGCGTTAATAAAAAGAAGGAACTCGTATTGTGTATGCGT...,ATGGTGTCCAAGGGGGAGGAGCTGTTCACCGGCGTGGTGCCCATCC...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,TTGTGTATGCGTTAATAAAAAGAAGGAACTCGTAAAAACTCAATGT...,ATGGTGTCCAAGGGGGAGGAGCTGTTCACCGGCGTGGTGCCCATCC...


In [40]:

experiment_UTR3_pair_exp.to_csv(f'/data1/genomics/yul/bioSPT2/data/interiem/processed/experiment_UTR3_pair_exp_cutoff{cutoff}.tsv', sep='\t')

#### CDS

In [ ]:

CDS bioSPT 项目实验反馈数据

In [42]:

experiment_CDS_exp = pd.read_excel('/home/yul/project/nas_project/bioSPT2/data/interiem/processed/mRNA全序列优化_实验验证结果_03192024.xlsx', header=1)

experiment_CDS_exp.head()

,No.,ID-Wet,Name-Wet,ID-Dry,Note-Dry,3-nt,5'UTR,CDS,3'UTR,5‘UTR-CDS,...,NA4.2,Test1.2,Test2.2,Test3.2,Test4.2,Ref1.2,Ref2.2,Ref3.2,Ref4.2,Ref seq (Name-Wet).2
0,1,CompleteSeq-1,pNeoCS-eGFPdegron-1,12.0,High_RSCU_cds1-1,AGG,GGCAAGGGCGAGTTCACCGGCGTGGTCCCAATTCTGGTCGAGCTCG...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,GCAGCGAAGGACGCTACGAAGCTCAGTCATTACCAGGAAGCTCCCT...,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,CompleteSeq-2,pNeoCS-eGFPdegron-2,13.0,High_RSCU_cds1-2,AGG,GGCAAGGGCGAGTTCACCGGCGTGGTCCCAATTCTGGTCGAGCTCG...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,GCTGGTCCCAATTCTGGTCGAGCTCGGTCAACGGACAGTTCTGTCC...,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,CompleteSeq-3,pNeoCS-eGFPdegron-3,14.0,High_RSCU_cds1-3,AGG,GGCAAGGGCGAGTTCACCGGCGTGGTCCCAATTCTGGTCGAGCTCG...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,GCCCGAAGGACGCTACGAAGCTCAGTCATTACCAGGAAGCTCCCTG...,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,CompleteSeq-4,pNeoCS-eGFPdegron-4,15.0,High_RSCU_cds2-1,AGG,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGCTCA...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,GTGGTCCCAATTCTGGTCGAGCTCACGGACAGTTCTGTACCAAACC...,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,CompleteSeq-5,pNeoCS-eGFPdegron-5,16.0,High_RSCU_cds2-2,AGG,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGCCGAGCTCG...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,TCGAGCTCGGACAGTTCTGTACCGTGCGGGCCAAGGGCGAAGCGTC...,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:

experiment_CDS_exp['mean_exp'] =  experiment_CDS_exp[
    [
     'Test1', 'Test2', 'Test3', 'Test4',
      'Test1.1', 'Test2.1','Test3.1', 'Test4.1',
     'Test1.2', 'Test2.2', 'Test3.2', 'Test4.2',
     ]
                    ].mean(axis=1)

experiment_CDS_exp = experiment_CDS_exp[ 
                   ~experiment_CDS_exp['mean_exp'].isnull()
                   ]

In [56]:


experiment_CDS_exp_agg = experiment_CDS_exp.groupby(by=['3-nt', "5'UTR", 'CDS', "3'UTR", '5‘UTR-CDS', 'CDS-3‘UTR']).agg(list)

agged_cols = list(experiment_CDS_exp_agg.columns)

experiment_CDS_exp_agg['gene_id'] = [f'gene_{g}' for g in range(experiment_CDS_exp_agg.shape[0])]

experiment_CDS_exp_agg_exploded = experiment_CDS_exp_agg.explode(agged_cols)

experiment_CDS_exp_agg_exploded = experiment_CDS_exp_agg_exploded.reset_index()

experiment_CDS_exp_agg_exploded.head()



,3-nt,5'UTR,CDS,3'UTR,5‘UTR-CDS,CDS-3‘UTR,No.,ID-Wet,Name-Wet,ID-Dry,...,Test2.2,Test3.2,Test4.2,Ref1.2,Ref2.2,Ref3.2,Ref4.2,Ref seq (Name-Wet).2,mean_exp,gene_id
0,AGG,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTGCTGTCGGGTG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,-,-,19,CompleteSeq-19,pNeoCS-PUMCH_S001-8,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4603.0,gene_0
1,AGG,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTTCTGTCGGGTG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,-,-,21,CompleteSeq-21,pNeoCS-PUMCH_S001-10,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5534.716667,gene_1
2,AGG,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTTCTGTCGGGTG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,-,-,30,CompleteSeq-30,pNeoCS-301_CMQI-8,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.628571,gene_2
3,AGG,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,ATGCGGGTGACGGCCCCTAGAACCCTGATTCTGCTTCTGTCGGGTG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,-,-,32,CompleteSeq-32,pNeoCS-301_CMQI-10,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.871429,gene_3
4,AGG,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,-,-,10,CompleteSeq-10,pNeoCS-eGFPdegron-10,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8808.75,gene_4


In [57]:


pairs = []
for i_index, i_row in experiment_CDS_exp_agg_exploded.iterrows():
    for j_index, j_row in experiment_CDS_exp_agg_exploded.iterrows():
    
        if i_row['mean_exp'] and j_row['mean_exp'] and \
            i_row['gene_id'] != j_row['gene_id'] and \
                str(Seq(i_row['CDS']).translate()) == str(Seq(j_row['CDS']).translate()):
            
            # print('...')
            pair = [i_row['gene_id'], j_row['gene_id']]
            if i_row['mean_exp']/j_row['mean_exp'] >= (1+cutoff):
                pair.append(1)
                
            elif i_row['mean_exp']/j_row['mean_exp'] <= 1/(1+cutoff):
                pair.append(-1)
                
            else:
                pair.append(0)
                
            pairs.append(pair)
            
            

In [58]:

experiment_CDS_pair_exp = pd.DataFrame(pairs, columns=['pair_A', 'pair_B', 'pair_label'])

experiment_CDS_pair_exp['pair_A_UTR5'] = experiment_CDS_pair_exp['pair_A'].apply(lambda x: experiment_CDS_exp_agg.reset_index().set_index('gene_id').reindex([x])["5'UTR"].values[0])
experiment_CDS_pair_exp['pair_A_UTR3'] = experiment_CDS_pair_exp['pair_A'].apply(lambda x: experiment_CDS_exp_agg.reset_index().set_index('gene_id').reindex([x])["3'UTR"].values[0])
experiment_CDS_pair_exp['pair_A_CDS'] = experiment_CDS_pair_exp['pair_A'].apply(lambda x: experiment_CDS_exp_agg.reset_index().set_index('gene_id').reindex([x])['CDS'].values[0])

experiment_CDS_pair_exp['pair_B_UTR5'] = experiment_CDS_pair_exp['pair_B'].apply(lambda x: experiment_CDS_exp_agg.reset_index().set_index('gene_id').reindex([x])["5'UTR"].values[0])
experiment_CDS_pair_exp['pair_B_UTR3'] = experiment_CDS_pair_exp['pair_B'].apply(lambda x: experiment_CDS_exp_agg.reset_index().set_index('gene_id').reindex([x])["3'UTR"].values[0])
experiment_CDS_pair_exp['pair_B_CDS'] = experiment_CDS_pair_exp['pair_B'].apply(lambda x: experiment_CDS_exp_agg.reset_index().set_index('gene_id').reindex([x])['CDS'].values[0])


experiment_CDS_pair_exp




,pair_A,pair_B,pair_label,pair_A_UTR5,pair_A_UTR3,pair_A_CDS,pair_B_UTR5,pair_B_UTR3,pair_B_CDS
0,gene_0,gene_1,0,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTGCTGTCGGGTG...,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTTCTGTCGGGTG...
1,gene_0,gene_7,-1,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTGCTGTCGGGTG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTCCTGCTGTCTGGAG...
2,gene_0,gene_8,0,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTGCTGTCGGGTG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTGCTGTCGGGTG...
3,gene_0,gene_9,0,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTGCTGTCGGGTG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTTCTGTCGGGTG...
4,gene_0,gene_15,-1,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTGCTGTCGGGTG...,CGGGTCACGGCCAACTTCTGTCGGGAGCCGGCGAGCGGTCGCCCTC...,GCGGCAGGCGGGAGGGCGAGCGGGAGGGAAGGGAGTGGCGGGTGCG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTTCTGTCGGGTG...
...,...,...,...,...,...,...,...,...,...
215,gene_25,gene_14,0,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGCTCA...,GTGGTCCCAATTCTGGTCGAGCTCACGGACAGTTCTGTACCAAACC...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...
216,gene_25,gene_21,1,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGCTCA...,GTGGTCCCAATTCTGGTCGAGCTCACGGACAGTTCTGTACCAAACC...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,GGCAAGGGCGAGTTCACCGGCGTGGTCCCAATTCTGGTCGAGCTCG...,GCAGCGAAGGACGCTACGAAGCTCAGTCATTACCAGGAAGCTCCCT...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...
217,gene_25,gene_22,1,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGCTCA...,GTGGTCCCAATTCTGGTCGAGCTCACGGACAGTTCTGTACCAAACC...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,GGCAAGGGCGAGTTCACCGGCGTGGTCCCAATTCTGGTCGAGCTCG...,GCTGGTCCCAATTCTGGTCGAGCTCGGTCAACGGACAGTTCTGTCC...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...
218,gene_25,gene_23,1,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGCTCA...,GTGGTCCCAATTCTGGTCGAGCTCACGGACAGTTCTGTACCAAACC...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGCCGAGCTCG...,TCGAGCTCGGACAGTTCTGTACCGTGCGGGCCAAGGGCGAAGCGTC...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...


In [60]:

experiment_CDS_pair_exp.to_csv(f'/data1/genomics/yul/bioSPT2/data/interiem/processed/experiment_CDS_pair_exp_cutoff{cutoff}.tsv', sep='\t')



#### CDS2

In [22]:

# 规范化fasta文件

def fasta2df(in_fna):
    """
    transform fasta file to dataframe
    """
    
    transcript_cdna = pd.DataFrame(columns=['gene_id', 'seq'])
    with open(in_fna, 'rt', encoding='utf-8') as f:
        all_line = f.readlines()

        for line in all_line:
            if line.startswith('>'):

                transcript_id =  line.lstrip('>').strip()

                transcript_cdna.loc[transcript_id, 'gene_id'] = transcript_id
                lines =""
            else:
                lines += line.strip()
                transcript_cdna.loc[transcript_id, 'seq'] = lines

    transcript_cdna['seq'] = transcript_cdna['seq'].apply(lambda x: x.upper().replace('U', 'T'))

    return transcript_cdna
                                               
                                               

##### Fluc

In [217]:



Fluc_CDS = fasta2df('data/eGFP_Fluc/Fluc_Exp.Collect.CDS.fasta')

Fluc_mRNA = fasta2df('data/eGFP_Fluc/Fluc_Exp.Collect.addUTR.fasta')

Fluc_mRNA.head()




,gene_id,seq
Neocura-Fluc_Exp01,Neocura-Fluc_Exp01,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...
Neocura-Fluc_Exp02,Neocura-Fluc_Exp02,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...
Neocura-Fluc_Exp03,Neocura-Fluc_Exp03,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...
Neocura-Fluc_Exp04,Neocura-Fluc_Exp04,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...
Neocura-Fluc_Exp05,Neocura-Fluc_Exp05,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...


In [218]:


def extract(row):
    
    CDS = Fluc_eGFP_CDS.reindex([row['gene_id']])['seq'].values[0]
    UTR5 = row['seq'].split(CDS)[0]
    UTR3 = row['seq'].split(CDS)[-1]
    
    # return pd.Series([UTR5, UTR5, UTR3])
    return pd.Series({
        'UTR5': UTR5, 'CDS':CDS, 'UTR3':UTR3
    })

    
Fluc_mRNA = Fluc_mRNA.apply(lambda row: extract(row), axis=1)

Fluc_mRNA




,UTR5,CDS,UTR3
Neocura-Fluc_Exp01,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCATTCTACC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...
Neocura-Fluc_Exp02,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCATTTTATC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...
Neocura-Fluc_Exp03,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCTTTCTATC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...
Neocura-Fluc_Exp04,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCTTTCTACC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...
Neocura-Fluc_Exp05,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCTCCCTTTTACC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...
Neocura-Fluc_Exp06,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCATTTTACC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...
Neocura-Fluc_Exp07,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCTTTCTACC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...
Neocura-Fluc_Exp08,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGAGGACGCCAAGAACATTAAGAAGGGCCCCGCCCCATTTTACC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...
Neocura-Fluc_Exp09,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCATTCTATC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...
Neocura-Fluc_Exp10,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCATTCTACC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...


In [219]:

Fluc_Exp = pd.read_excel('data/eGFP_Fluc/Fluc_Exp.Result.Normalized.xlsx')
Fluc_Exp['gene_id'] = Fluc_Exp['ExpID']
Fluc_Exp.head()


,No.,Value,ExpID,TaowaScore,TaowaRank,StemCount,StemRank,三等分,TaowaScore_7500,gene_id
0,No.11,1.614245,Neocura-Fluc_Exp11,14172,1.0,616,6.0,High,High,Neocura-Fluc_Exp11
1,No.28,1.055267,Neocura-Fluc_Exp28,14104,2.0,678,2.5,High,High,Neocura-Fluc_Exp28
2,No.17,1.511719,Neocura-Fluc_Exp17,9970,3.0,604,9.5,High,High,Neocura-Fluc_Exp17
3,No.23,1.672987,Neocura-Fluc_Exp23,9362,4.0,700,1.0,High,High,Neocura-Fluc_Exp23
4,No.33,1.479376,Neocura-Fluc_Exp33,9198,5.0,642,4.0,High,High,Neocura-Fluc_Exp33


##### eGFP

In [220]:


eGFP_CDS = fasta2df('data/eGFP_Fluc/eGFP-Tested.Collect.CDS.fasta')

eGFP_mRNA = fasta2df('data/eGFP_Fluc/eGFP-Tested.Collect.addUTR.fasta')

    
eGFP_mRNA = eGFP_mRNA.apply(lambda row: extract(row), axis=1)


eGFP_mRNA


,UTR5,CDS,UTR3
eGFP-degron_Test2013,GGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGTGAGCAAGGGCGAGGAGCTGTTTACAGGCGTGGTGCCCATCC...,GAGCTCCTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCG...
eGFP-degron_Test3690,GGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GAGCTCCTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCG...
eGFP-degron_Test3006,GGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GAGCTCCTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCG...
eGFP-degron_Test2124,GGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGTGAGCAAGGGCGAGGAGCTGTTTACAGGAGTGGTGCCCATCC...,GAGCTCCTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCG...
eGFP-degron_Test3768,GGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GAGCTCCTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCG...
...,...,...,...
pNeoCS-eGFPdegron-7,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...
pNeoCS-eGFPdegron-8,AGGAAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTT...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...
pNeoCS-eGFPdegron-9,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...
pNeoCS-eGFPdegron-10,AGGAAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTT...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...


In [213]:


Fluc_eGFP_CDS = pd.concat([Fluc_CDS, eGFP_CDS])

Fluc_eGFP_CDS



,gene_id,seq
Neocura-Fluc_Exp01,Neocura-Fluc_Exp01,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCATTCTACC...
Neocura-Fluc_Exp02,Neocura-Fluc_Exp02,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCATTTTATC...
Neocura-Fluc_Exp03,Neocura-Fluc_Exp03,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCTTTCTATC...
Neocura-Fluc_Exp04,Neocura-Fluc_Exp04,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCTTTCTACC...
Neocura-Fluc_Exp05,Neocura-Fluc_Exp05,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCTCCCTTTTACC...
...,...,...
pNeoCS-eGFPdegron-7,pNeoCS-eGFPdegron-7,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...
pNeoCS-eGFPdegron-8,pNeoCS-eGFPdegron-8,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...
pNeoCS-eGFPdegron-9,pNeoCS-eGFPdegron-9,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...
pNeoCS-eGFPdegron-10,pNeoCS-eGFPdegron-10,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...


In [87]:
import yaml
with open('data/eGFP_Fluc/eGFP-degron_CDS-Exp_New.yaml') as fn:
    eGFP_Exp = pd.DataFrame.from_dict(
        yaml.load(fn, Loader=yaml.FullLoader), orient='index', columns=['Value'])
    
eGFP_Exp['gene_id'] = eGFP_Exp.index
eGFP_Exp.head()



,Value,gene_id
High_RSCU_high_struc_3,1.000000,High_RSCU_high_struc_3
Mid_RSCU_high_struc_1,0.527020,Mid_RSCU_high_struc_1
Mid_RSCU_high_struc_2,0.488853,Mid_RSCU_high_struc_2
Mid_RSCU_high_struc_3,0.408157,Mid_RSCU_high_struc_3
Mid_RSCU_high_struc_4,0.443380,Mid_RSCU_high_struc_4


In [221]:

eGFP_Fluc_Exp = pd.concat([Fluc_Exp, eGFP_Exp], axis=0)

eGFP_Fluc_Exp


eGFP_Fluc_fasta = pd.concat([Fluc_mRNA, eGFP_mRNA], axis=0)


eGFP_Fluc_fasta.head()


,UTR5,CDS,UTR3
Neocura-Fluc_Exp01,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCATTCTACC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...
Neocura-Fluc_Exp02,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCATTTTATC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...
Neocura-Fluc_Exp03,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCTTTCTATC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...
Neocura-Fluc_Exp04,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCTTTCTACC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...
Neocura-Fluc_Exp05,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCTCCCTTTTACC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...


In [130]:

eGFP_Fluc_Exp['gene_id']



0                       Neocura-Fluc_Exp11
1                       Neocura-Fluc_Exp28
2                       Neocura-Fluc_Exp17
3                       Neocura-Fluc_Exp23
4                       Neocura-Fluc_Exp33
                              ...         
pNeoCS-eGFPdegron-5    pNeoCS-eGFPdegron-5
pNeoCS-eGFPdegron-6    pNeoCS-eGFPdegron-6
pNeoCS-eGFPdegron-7    pNeoCS-eGFPdegron-7
pNeoCS-eGFPdegron-8    pNeoCS-eGFPdegron-8
pNeoCS-eGFPdegron-9    pNeoCS-eGFPdegron-9
Name: gene_id, Length: 126, dtype: object

In [182]:

eGFP_Exp['gene_id'].apply(lambda x: eGFP_mRNA.reindex([x])['CDS'].values)




High_RSCU_high_struc_3    [ATGGTGTCCAAGGGGGAGGAGCTGTTCACCGGCGTGGTGCCCATC...
Mid_RSCU_high_struc_1     [ATGGTCTCGAAGGGCGAGGAGTTGTTCACCGGGGTGGTGCCGATC...
Mid_RSCU_high_struc_2     [ATGGTCTCGAAGGGCGAGGAGCTGTTCACGGGAGTCGTCCCGATC...
Mid_RSCU_high_struc_3                                   [ATGGTCTCGAAGGGTGA]
Mid_RSCU_high_struc_4     [ATGGTCTCGAAGGGAGAGGAGCTCTTCACGGGGGTCGTCCCGATC...
                                                ...                        
pNeoCS-eGFPdegron-5                                               [ATGGTGA]
pNeoCS-eGFPdegron-6                                               [ATGGTGA]
pNeoCS-eGFPdegron-7                                               [ATGGTGA]
pNeoCS-eGFPdegron-8                                               [ATGGTGA]
pNeoCS-eGFPdegron-9                                               [ATGGTGA]
Name: gene_id, Length: 88, dtype: object

In [222]:


eGFP_Fluc_Exp['CDS'] = eGFP_Fluc_Exp['gene_id'].apply(lambda x: eGFP_Fluc_fasta.reindex([x])['CDS'].values[0])

eGFP_Fluc_Exp['CDS'].tail()



pNeoCS-eGFPdegron-5    ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...
pNeoCS-eGFPdegron-6    ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...
pNeoCS-eGFPdegron-7    ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...
pNeoCS-eGFPdegron-8    ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...
pNeoCS-eGFPdegron-9    ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...
Name: CDS, dtype: object

In [231]:

eGFP_Fluc_Exp['Value']



0                      1.614245
1                      1.055267
2                      1.511719
3                      1.672987
4                      1.479376
                         ...   
pNeoCS-eGFPdegron-5    0.100000
pNeoCS-eGFPdegron-6    1.200000
pNeoCS-eGFPdegron-7    1.100000
pNeoCS-eGFPdegron-8    1.300000
pNeoCS-eGFPdegron-9    1.500000
Name: Value, Length: 126, dtype: float64

In [236]:

cutoff = 0

pairs = []
for i_index, i_row in eGFP_Fluc_Exp.iterrows():
    for j_index, j_row in eGFP_Fluc_Exp.iterrows():
    
        if i_row['Value'] and j_row['Value'] and \
            i_row['gene_id'] != j_row['gene_id'] and \
                str(Seq(i_row['CDS']).translate()) == str(Seq(j_row['CDS']).translate()):
            
            pair = [i_row['gene_id'], j_row['gene_id']]
            if i_row['Value']/j_row['Value'] >= (1+cutoff):
                pair.append(1)
                
            elif i_row['Value']/j_row['Value'] <= 1/(1+cutoff):
                pair.append(-1)
                
            else:
                pair.append(0)
                
                
            pair.append(i_row['Value']/j_row['Value'])
                
            pairs.append(pair)
                  
            
            

In [237]:


experiment_CDS_pair_exp = pd.DataFrame(pairs, columns=['pair_A', 'pair_B',  'pair_label', 'pair_diff'])

experiment_CDS_pair_exp['pair_A_UTR5'] = experiment_CDS_pair_exp['pair_A'].apply(lambda x: eGFP_Fluc_fasta.reindex([x])["UTR5"].values[0])
experiment_CDS_pair_exp['pair_A_UTR3'] = experiment_CDS_pair_exp['pair_A'].apply(lambda x: eGFP_Fluc_fasta.reindex([x])["UTR3"].values[0])
experiment_CDS_pair_exp['pair_A_CDS'] = experiment_CDS_pair_exp['pair_A'].apply(lambda x: eGFP_Fluc_fasta.reindex([x])['CDS'].values[0])

experiment_CDS_pair_exp['pair_B_UTR5'] = experiment_CDS_pair_exp['pair_B'].apply(lambda x: eGFP_Fluc_fasta.reindex([x])["UTR5"].values[0])
experiment_CDS_pair_exp['pair_B_UTR3'] = experiment_CDS_pair_exp['pair_B'].apply(lambda x: eGFP_Fluc_fasta.reindex([x])["UTR3"].values[0])
experiment_CDS_pair_exp['pair_B_CDS'] = experiment_CDS_pair_exp['pair_B'].apply(lambda x: eGFP_Fluc_fasta.reindex([x])['CDS'].values[0])


experiment_CDS_pair_exp




,pair_A,pair_B,pair_label,pair_diff,pair_A_UTR5,pair_A_UTR3,pair_A_CDS,pair_B_UTR5,pair_B_UTR3,pair_B_CDS
0,Neocura-Fluc_Exp11,Neocura-Fluc_Exp28,1,1.529703,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCATTCTATC...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAATATCAAGAAGGGCCCAGCCCCATTCTATC...
1,Neocura-Fluc_Exp11,Neocura-Fluc_Exp17,1,1.067821,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCATTCTATC...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATTAAGAAGGGCCCCGCTCCCTTCTATC...
2,Neocura-Fluc_Exp11,Neocura-Fluc_Exp23,-1,0.964888,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCATTCTATC...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCATTCTACC...
3,Neocura-Fluc_Exp11,Neocura-Fluc_Exp33,1,1.091166,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCATTCTATC...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCTTTCTACC...
4,Neocura-Fluc_Exp11,Neocura-Fluc_Exp04,1,1.040731,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCATTCTATC...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCTTTCTACC...
...,...,...,...,...,...,...,...,...,...,...
8809,pNeoCS-eGFPdegron-9,pNeoCS-eGFPdegron-4,1,1.500000,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,AGGGGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGC...,GTGGTCCCAATTCTGGTCGAGCTCACGGACAGTTCTGTACCAAACC...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...
8810,pNeoCS-eGFPdegron-9,pNeoCS-eGFPdegron-5,1,15.000000,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,AGGGGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGCCGAGC...,TCGAGCTCGGACAGTTCTGTACCGTGCGGGCCAAGGGCGAAGCGTC...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...
8811,pNeoCS-eGFPdegron-9,pNeoCS-eGFPdegron-6,1,1.250000,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,AGGGGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGC...,GTGGTCCCAATTCTGGTCGAGCTCACGGACAGTTCTGTACCAAACC...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...
8812,pNeoCS-eGFPdegron-9,pNeoCS-eGFPdegron-7,1,1.363636,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...


In [230]:

experiment_CDS_pair_exp['pair_label'].value_counts()



 1    3020
-1    3020
 0    2774
Name: pair_label, dtype: int64

In [238]:

experiment_CDS_pair_exp.to_csv(f'data/interim/experiment_CDS2_pair_exp_cutoff{cutoff}.tsv', sep='\t')




####  合并去重

##### cellines_pair_exp

In [71]:

cellines_pair_exp = pd.read_csv('/home/yul/project/nas_project/bioSPT2/data/interiem/processed/HEK_ESC_data_db_0.8_pairs_expression.tsv', sep='\t')

cellines_pair_exp.head()



,Unnamed: 0,gene1,gene2,ESC,TE,iBAQ,pair_label
0,0,ENSG00000160336,ENSG00000213799,0,-1,-1,-1
1,1,ENSG00000160336,ENSG00000196417,0,0,0,0
2,2,ENSG00000213799,ENSG00000196417,-1,1,0,0
3,3,ENSG00000072110,ENSG00000130402,0,-1,0,-1
4,4,ENSG00000178935,ENSG00000269343,0,1,0,1


In [78]:

cellines_pair_exp['pair_A'] = cellines_pair_exp['gene1']
cellines_pair_exp['pair_B'] = cellines_pair_exp['gene2']

cellines_pair_exp = cellines_pair_exp[[ 'pair_A', 'pair_B',  'pair_label']]
cellines_pair_exp

,pair_A,pair_B,pair_label
0,ENSG00000160336,ENSG00000213799,-1
1,ENSG00000160336,ENSG00000196417,0
2,ENSG00000213799,ENSG00000196417,0
3,ENSG00000072110,ENSG00000130402,-1
4,ENSG00000178935,ENSG00000269343,1
...,...,...,...
59,ENSG00000270276,ENSG00000277157,0
60,ENSG00000270276,ENSG00000278637,0
61,ENSG00000276180,ENSG00000277157,-1
62,ENSG00000276180,ENSG00000278637,-1


In [79]:

cellines_pair_exp = cellines_pair_exp[cellines_pair_exp['pair_label'] != 0]

cellines_pair_exp.shape

In [72]:
human_ts.head()

,Transcript,gene_id,name,seq,transcript_id,gene_id_raw,gene_name,UTR5_regs,CDS_regs,UTR3_regs,UTR5,CDS,UTR3,Esemble
0,ENST00000373020,ENSG00000000003.15,>ENST00000373020.9|ENSG00000000003.15|OTTHUMG0...,AGTTGTGGACGCTCGTAAGTTTTCGGCAGTTTCCGGGGAGACTCGG...,ENST00000373020.9,ENST00000373020.9,TSPAN6,UTR5:1-112,CDS:113-850,UTR3:851-3768,AGTTGTGGACGCTCGTAAGTTTTCGGCAGTTTCCGGGGAGACTCGG...,ATGGCGTCCCCGTCTCGGAGACTGCAGACTAAACCAGTCATTACTT...,CCCAATGTATCTGTGGGCCTATTCCTCTCTACCTTTAAGGACATTT...,ENSG00000000003
1,ENST00000373031,ENSG00000000005.6,>ENST00000373031.5|ENSG00000000005.6|OTTHUMG00...,AGCCGACTCACTTGCAACTCCACCTCAGCAGTGGTCTCTCAGTCCT...,ENST00000373031.5,ENST00000373031.5,TNMD,UTR5:1-83,CDS:84-1037,UTR3:1038-1205,AGCCGACTCACTTGCAACTCCACCTCAGCAGTGGTCTCTCAGTCCT...,ATGGCAAAGAATCCTCCAGAGAATTGTGAAGACTGTCACATTCTAA...,TAGGAGGTTTGAGCTCAAATGCTTAAACTGCTGGCAACATATAATA...,ENSG00000000005
2,ENST00000371588,ENSG00000000419.14,>ENST00000371588.10|ENSG00000000419.14|OTTHUMG...,AGTTCCGCCATGGCCTCCTTGGAAGTCAGTCGTAGTCCTCGCAGGT...,ENST00000371588.10,ENST00000371588.10,DPM1,UTR5:1-9,CDS:10-792,UTR3:793-1054,AGTTCCGCC,ATGGCCTCCTTGGAAGTCAGTCGTAGTCCTCGCAGGTCTCGGCGGG...,AAGAAAGATACTCATTTATAGTTACGTTCATTTCAGGTTAAACATG...,ENSG00000000419
3,ENST00000367772,ENSG00000000457.14,>ENST00000367772.8|ENSG00000000457.14|OTTHUMG0...,TGTCCCGTTTCCGGACCCGTCTCTATGGTGTAGGAGAAACCCGGCC...,ENST00000367772.8,ENST00000367772.8,SCYL3,UTR5:1-222,CDS:223-2451,UTR3:2452-3090,TGTCCCGTTTCCGGACCCGTCTCTATGGTGTAGGAGAAACCCGGCC...,ATGGGATCAGAGAACAGTGCTTTAAAGAGCTATACACTGAGAGAAC...,CAATAGATGTGAGTTAAACTTTAGGAAAAAGGATTCCCTTTTTTTA...,ENSG00000000457
4,ENST00000413811,ENSG00000000460.17,>ENST00000413811.3|ENSG00000000460.17|OTTHUMG0...,ACATTCTTGATTGGTCTCATGCCTTTGTGGTTGTAAATGTGCCTGG...,ENST00000413811.3,ENST00000413811.3,C1orf112,UTR5:1-1018,CDS:1019-2611,UTR3:2612-2661,ACATTCTTGATTGGTCTCATGCCTTTGTGGTTGTAAATGTGCCTGG...,ATGCAGGTGGTTTTGGACAGTAAATTAGACCTGCCATGTGAACTGC...,AACTTATCACTAGGCAGAACTGGGTTTGATGCTTTGTCAACTGAAA...,ENSG00000000460


In [101]:


cellines_pair_exp['pair_A_UTR5'] = cellines_pair_exp['pair_A'].apply(lambda x: human_ts.reset_index().set_index('Esemble').reindex([x])["UTR5"].values[0])
cellines_pair_exp['pair_A_UTR3'] = cellines_pair_exp['pair_A'].apply(lambda x: human_ts.reset_index().set_index('Esemble').reindex([x])["UTR3"].values[0])
cellines_pair_exp['pair_A_CDS'] = cellines_pair_exp['pair_A'].apply(lambda x: human_ts.reset_index().set_index('Esemble').reindex([x])['CDS'].values[0])

cellines_pair_exp['pair_B_UTR5'] = cellines_pair_exp['pair_B'].apply(lambda x: human_ts.reset_index().set_index('Esemble').reindex([x])["UTR5"].values[0])
cellines_pair_exp['pair_B_UTR3'] = cellines_pair_exp['pair_B'].apply(lambda x: human_ts.reset_index().set_index('Esemble').reindex([x])["UTR3"].values[0])
cellines_pair_exp['pair_B_CDS'] = cellines_pair_exp['pair_B'].apply(lambda x: human_ts.reset_index().set_index('Esemble').reindex([x])['CDS'].values[0])



/tmp/ipykernel_126156/2100682807.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cellines_pair_exp['pair_A_UTR5'] = cellines_pair_exp['pair_A'].apply(lambda x: human_ts.reset_index().set_index('Esemble').reindex([x])["UTR5"].values[0])
/tmp/ipykernel_126156/2100682807.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cellines_pair_exp['pair_A_UTR3'] = cellines_pair_exp['pair_A'].apply(lambda x: human_ts.reset_index().set_index('Esemble').reindex([x])["UTR3"].values[0])
/tmp/ipykernel_126156/2100682807

In [102]:

cellines_pair_exp['pair_B_UTR3'].isnull().sum()

0

In [116]:
# 生成prompts

prompts = []
for index, row in cellines_pair_exp.iterrows():
    
    for item in ['pair_A_CDS', 'pair_B_CDS']:
        
        ppt = str(Seq(row[item]).translate())
        for jtem in ['UTR5', 'UTR3','CDS']:
            
            pairA_token = row[f'pair_A_{jtem}']
            pairB_token = row[f'pair_B_{jtem}']
            label = row['pair_label']
            g_type=jtem
            origin = 'cellines_cluster'
            
            prompts.append([ppt, pairA_token, pairB_token, g_type, origin, label])


In [118]:

cellines_prompts = pd.DataFrame(prompts, columns=['prompt', 'pair_A', 'pair_B', 'seq_type', 'origin', 'pair_label'])

cellines_prompts = cellines_prompts.drop_duplicates(subset=['prompt', 'pair_A', 'pair_B', 'seq_type'])

cellines_prompts = cellines_prompts[ 
                                    cellines_prompts['pair_A']  != cellines_prompts['pair_B']
                                    ]
  
cellines_prompts

,prompt,pair_A,pair_B,seq_type,origin,pair_label
0,MAFSQGLLTFRDVAIEFSQEEWKCLDPAQRTLYRDVMLENYRNLVS...,ATGGAGTTTCGCTCTTGTTGCTCAGGCTGGAGTGCAATGGTGTGAT...,AAACCCGGAAGCGGATCGCATGGAGTGATGGTCCCACCGCAGCGCC...,UTR5,cellines_cluster,-1
1,MAFSQGLLTFRDVAIEFSQEEWKCLDPAQRTLYRDVMLENYRNLVS...,TGAATGTGGTGAGGTTTTTAATCAACAAGCACACCTTGCAGGTCAT...,AAATATGAAGAATGTGACAAAGTTTACAGTTGTAAATCAAGTCTTG...,UTR3,cellines_cluster,-1
2,MAFSQGLLTFRDVAIEFSQEEWKCLDPAQRTLYRDVMLENYRNLVS...,ATGGCTTTTTCTCAGGGTCTATTGACATTCAGGGATGTGGCCATAG...,ATGGCTCTTTCTCAGGGTCTATTGACATTCAGGGATGTGGCCATAG...,CDS,cellines_cluster,-1
3,MALSQGLLTFRDVAIEFSQEEWKCLDPAQRTLYRDVMLENYRNLVS...,ATGGAGTTTCGCTCTTGTTGCTCAGGCTGGAGTGCAATGGTGTGAT...,AAACCCGGAAGCGGATCGCATGGAGTGATGGTCCCACCGCAGCGCC...,UTR5,cellines_cluster,-1
4,MALSQGLLTFRDVAIEFSQEEWKCLDPAQRTLYRDVMLENYRNLVS...,TGAATGTGGTGAGGTTTTTAATCAACAAGCACACCTTGCAGGTCAT...,AAATATGAAGAATGTGACAAAGTTTACAGTTGTAAATCAAGTCTTG...,UTR3,cellines_cluster,-1
...,...,...,...,...,...,...
229,MSGRGKGGKGLGKGGAKRHRKVLRDNIQGITKPAIRRLARRGGVKR...,ATGGCATTTTGAAGCCCAGTCATTCTCTAAAAAGGCCCTTTTTAGG...,GCTTACCTCTACAGTACACTACCGCAAAACCAACGGCCCTTTTCAG...,UTR3,cellines_cluster,-1
230,MSGRGKGGKGLGKGGAKRHRKVLRDNIQGITKPAIRRLARRGGVKR...,ATGTCAGGACGCGGCAAAGGAGGTAAGGGCCTGGGGAAAGGGGGTG...,ATGTCTGGCCGCGGTAAGGGCGGAAAGGGTCTAGGTAAGGGTGGCG...,CDS,cellines_cluster,-1
234,MSGRGKGGKGLGKGGAKRHRKVLRDNIQGITKPAIRRLARRGGVKR...,AGACCTTTGTTCTCTGACCACTTGATA,ACTTGCTCTTGGTTCACTTCTTGGGAAGTC,UTR5,cellines_cluster,-1
235,MSGRGKGGKGLGKGGAKRHRKVLRDNIQGITKPAIRRLARRGGVKR...,ATGGCATTTTGAAGCCCAGTCATTCTCTAAAAAGGCCCTTTTTAGG...,GGTTGCTGATTTCTCCACAGCTTGCATTTCTGAACCAAAGGCCCTT...,UTR3,cellines_cluster,-1


##### pair_utr5_exp

In [239]:

cutoff = 0

In [242]:

pair_utr5_exp = pd.read_csv(f'data/experiment_UTR5_pair_exp_cutoff{cutoff}.tsv', sep='\t')
pair_utr5_exp = pair_utr5_exp[pair_utr5_exp['pair_label'] != 0]
pair_utr5_exp.head()


,Unnamed: 0,pair_A,pair_B,pair_label,pair_diff,pair_A_UTR5,pair_A_UTR3,pair_A_CDS,pair_B_UTR5,pair_B_UTR3,pair_B_CDS
0,0,gene_0,gene_52,1,3.300000,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GGGAAATAAGAGAGAAAAGAAGAGTAAGAAGAAATATAAGACCCCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...
1,1,gene_0,gene_56,1,1.650000,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...
2,2,gene_0,gene_60,1,1.650000,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GGGAATCAACACAGACACCAGACCACCAGCACCAGAACACACAGAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...
3,3,gene_0,gene_64,1,3.666667,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GGGAATCACCACCAGAACACACACACAGAAACACACAGACACCAGA...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...
4,4,gene_0,gene_68,1,5.500000,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,GGGAATTCGTCGACTCAGGAACTATTCTTCTGGTCCCCACAGACTC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...


In [277]:

all([row[f'pair_A_{tem}']==row[f'pair_B_{tem}'] or row[f'pair_A_{tem}'][4:-4]==row[f'pair_B_{tem}'][4:-4] for tem in ['UTR5', 'UTR3','CDS'] if tem!=jtem])

# jtem

# row[f'pair_A_UTR5']==row[f'pair_B_UTR5']



False

In [287]:
# 生成prompts

prompts = []
for index, row in pair_utr5_exp.iterrows():
    
    for item in ['pair_A_CDS', 'pair_B_CDS']:
        
        ppt = str(Seq(row[item]).translate())
        
        for jtem in [ 'mRNA', 'UTR5','CDS', 'UTR3']:
            
            if jtem == 'mRNA':
                pairA_token = '_'.join([row[f'pair_A_UTR5'], row[f'pair_A_CDS'], row[f'pair_A_UTR3']  ])
                pairB_token = '_'.join([row[f'pair_B_UTR5'], row[f'pair_B_CDS'], row[f'pair_B_UTR3']  ])
                exact_diff = (pairA_token == pairB_token)
                roughly_diff = (pairA_token == pairB_token) 
            else:
                exact_diff = all([row[f'pair_A_{jtem}']==row[f'pair_B_{jtem}'] for tem in ['UTR5', 'UTR3','CDS'] if tem!=jtem])
                roughly_diff = all([row[f'pair_A_{tem}']==row[f'pair_B_{tem}'] or row[f'pair_A_{tem}'][4:-4]==row[f'pair_B_{tem}'][4:-4] for tem in ['UTR5', 'UTR3','CDS'] if tem!=jtem])
                pairA_token = row[f'pair_A_{jtem}']
                pairB_token = row[f'pair_B_{jtem}']
                
            label = row['pair_label']
            value = row['pair_diff']
            g_type=jtem
            origin = 'utr5_exp'
            
            prompts.append([ppt, pairA_token, pairB_token, g_type, origin, label, value, exact_diff, roughly_diff])
            
            
            

In [289]:
UTR5_prompts_raw[UTR5_prompts_raw['seq_type']=='UTR5']['exact_diff'].value_counts()

# prompts


False    17560
Name: exact_diff, dtype: int64

In [288]:

UTR5_prompts = pd.DataFrame(prompts, columns=['prompt', 'pair_A', 'pair_B', 'seq_type', 'origin', 'pair_label', 'pair_diff', 'exact_diff', 'roughly_diff'])

UTR5_prompts_raw = UTR5_prompts

UTR5_prompts = UTR5_prompts.drop_duplicates(subset=['prompt', 'pair_A', 'pair_B', 'seq_type'])

UTR5_prompts = UTR5_prompts[ 
                                    UTR5_prompts['pair_A']  != UTR5_prompts['pair_B']
                                    ]

UTR5_prompts



,prompt,pair_A,pair_B,seq_type,origin,pair_label,pair_diff,exact_diff,roughly_diff
0,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,GGGAAATAAGAGAGAAAAGAAGAGTAAGAAGAAATATAAGACCCCG...,mRNA,utr5_exp,1,3.30,False,False
1,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,GGGAAATAAGAGAGAAAAGAAGAGTAAGAAGAAATATAAGACCCCG...,UTR5,utr5_exp,1,3.30,False,True
8,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,GGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,mRNA,utr5_exp,1,1.65,False,False
9,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,GGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,UTR5,utr5_exp,1,1.65,False,True
16,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,GGGAATCAACACAGACACCAGACCACCAGCACCAGAACACACAGAC...,mRNA,utr5_exp,1,1.65,False,False
...,...,...,...,...,...,...,...,...,...
70200,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GGGTCTCAACACAACATATACAAAACAAACGAATCTCAAGCAATCA...,GGGATTCCCCCACCACCAGAGCACCAGTCCCGCAGTCGGCGTCCAG...,mRNA,utr5_exp,1,1.05,False,False
70208,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GGGTCTCAACACAACATATACAAAACAAACGAATCTCAAGCAATCA...,GGGATTCCCCGCCTCCTCCTGTCCCGCAGTCGGCGTCCAGCGGCTC...,mRNA,utr5_exp,-1,0.84,False,False
70216,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GGGTCTCAACACAACATATACAAAACAAACGAATCTCAAGCAATCA...,GGGGCCACCAAGTTGTTACATGAAATCTGCAGTTTCATAATTTCCG...,mRNA,utr5_exp,1,3.15,False,False
70224,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GGGTCTCAACACAACATATACAAAACAAACGAATCTCAAGCAATCA...,GGGGCCAGCAGAAGTTGTTACAAATCTGCAGTTTCATTTCCGTGGG...,mRNA,utr5_exp,1,6.30,False,False


#####  pair_utr3_exp

In [290]:

pair_utr3_exp = pd.read_csv(f'data/experiment_UTR3_pair_exp_cutoff{cutoff}.tsv', sep='\t')

pair_utr3_exp = pair_utr3_exp[pair_utr3_exp['pair_label'] != 0]
pair_utr3_exp.head()

,Unnamed: 0,pair_A,pair_B,pair_label,pair_diff,pair_A_UTR5,pair_A_UTR3,pair_A_CDS,pair_B_UTR5,pair_B_UTR3,pair_B_CDS
0,0,gene_0,gene_1,1,1.065332,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ACTAATACTGGTCTGCGGCAGCTAAATAAGATCAATAAATTAATTC...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ACTGTACCATTTACCACTGTGACTAAATAAAGGGTGTGTCTGTTGT...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...
1,1,gene_0,gene_2,-1,0.418709,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ACTAATACTGGTCTGCGGCAGCTAAATAAGATCAATAAATTAATTC...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,CCCCCAATATCTGCTCCTCTGGTTTGGCTTCCCTCAGAGCCTGGGT...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...
2,2,gene_0,gene_3,-1,0.434997,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ACTAATACTGGTCTGCGGCAGCTAAATAAGATCAATAAATTAATTC...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,CCCCGTCTGCTCACCTGCAAAATAAAAGGCTTTTCGCCCCTTGTCC...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...
3,3,gene_0,gene_4,-1,0.433300,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ACTAATACTGGTCTGCGGCAGCTAAATAAGATCAATAAATTAATTC...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,CCCCGTCTGCTCCTCTAGGCTGGCCCCTTGTCCACCCCTCTCCACA...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...
4,4,gene_0,gene_5,-1,0.421146,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,ACTAATACTGGTCTGCGGCAGCTAAATAAGATCAATAAATTAATTC...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,CCCCTTCTGTTCTGCTCCTCTAATAAAGGCTCTGATTTTCCCCCAT...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTGCTCCTGTCTGGAG...


In [301]:

# 生成prompts

prompts = []
for index, row in pair_utr3_exp.iterrows():
    
    for item in ['pair_A_CDS', 'pair_B_CDS']:
        
        ppt = str(Seq(row[item]).translate())
        
        for jtem in [ 'mRNA', 'UTR5','CDS', 'UTR3']:
            
            if jtem == 'mRNA':
                pairA_token = '_'.join([row[f'pair_A_UTR5'], row[f'pair_A_CDS'], row[f'pair_A_UTR3']  ])
                pairB_token = '_'.join([row[f'pair_B_UTR5'], row[f'pair_B_CDS'], row[f'pair_B_UTR3']  ])
                exact_diff = (pairA_token == pairB_token)
                roughly_diff = (pairA_token == pairB_token) 
            else:
                exact_diff = all([row[f'pair_A_{jtem}']==row[f'pair_B_{jtem}'] for tem in ['UTR5', 'UTR3','CDS'] if tem!=jtem])
                roughly_diff = all([row[f'pair_A_{tem}']==row[f'pair_B_{tem}'] or row[f'pair_A_{tem}'][4:-4]==row[f'pair_B_{tem}'][4:-4] for tem in ['UTR5', 'UTR3','CDS'] if tem!=jtem])
                pairA_token = row[f'pair_A_{jtem}']
                pairB_token = row[f'pair_B_{jtem}']
                
            label = row['pair_label']
            value = row['pair_diff']
            g_type=jtem
            origin = 'utr3_exp'
            
            prompts.append([ppt, pairA_token, pairB_token, g_type, origin, label, value, exact_diff, roughly_diff])
            
            
            
            
            

In [302]:

UTR3_prompts = pd.DataFrame(prompts, columns=['prompt', 'pair_A', 'pair_B', 'seq_type', 'origin', 'pair_label', 'pair_diff', 'exact_diff', 'roughly_diff'])

UTR3_prompts_raw = UTR3_prompts

UTR3_prompts = UTR3_prompts.drop_duplicates(subset=['prompt', 'pair_A', 'pair_B', 'seq_type'])

UTR3_prompts = UTR3_prompts[ 
                                    UTR3_prompts['pair_A']  != UTR3_prompts['pair_B']
                                    ]

UTR3_prompts


,prompt,pair_A,pair_B,seq_type,origin,pair_label,pair_diff,exact_diff,roughly_diff
0,MRVTAPRTLILLLSGALALTETWAGSGGGGSGGCSVDNCRLFISGI...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,mRNA,utr3_exp,1,1.065332,False,False
3,MRVTAPRTLILLLSGALALTETWAGSGGGGSGGCSVDNCRLFISGI...,ACTAATACTGGTCTGCGGCAGCTAAATAAGATCAATAAATTAATTC...,ACTGTACCATTTACCACTGTGACTAAATAAAGGGTGTGTCTGTTGT...,UTR3,utr3_exp,1,1.065332,False,True
8,MRVTAPRTLILLLSGALALTETWAGSGGGGSGGCSVDNCRLFISGI...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,mRNA,utr3_exp,-1,0.418709,False,False
11,MRVTAPRTLILLLSGALALTETWAGSGGGGSGGCSVDNCRLFISGI...,ACTAATACTGGTCTGCGGCAGCTAAATAAGATCAATAAATTAATTC...,CCCCCAATATCTGCTCCTCTGGTTTGGCTTCCCTCAGAGCCTGGGT...,UTR3,utr3_exp,-1,0.418709,False,True
16,MRVTAPRTLILLLSGALALTETWAGSGGGGSGGCSVDNCRLFISGI...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,mRNA,utr3_exp,-1,0.434997,False,False
...,...,...,...,...,...,...,...,...,...
54728,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,mRNA,utr3_exp,-1,0.472478,False,False
54736,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,mRNA,utr3_exp,-1,0.726330,False,False
54744,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,mRNA,utr3_exp,1,1.318426,False,False
54752,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,mRNA,utr3_exp,-1,0.653769,False,False


##### pair_CDS_exp

In [294]:

pair_CDS_exp = pd.read_csv(f'data/experiment_CDS_pair_exp_cutoff{cutoff}.tsv', sep='\t')
pair_CDS_exp = pair_CDS_exp[pair_CDS_exp['pair_label'] != 0]
pair_CDS_exp.head()



,Unnamed: 0,pair_A,pair_B,pair_label,pair_diff,pair_A_UTR5,pair_A_UTR3,pair_A_CDS,pair_B_UTR5,pair_B_UTR3,pair_B_CDS
0,0,gene_0,gene_1,-1,0.831660,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTGCTGTCGGGTG...,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTTCTGTCGGGTG...
1,1,gene_0,gene_7,-1,0.751856,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTGCTGTCGGGTG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTCCTGCTGTCTGGAG...
2,2,gene_0,gene_8,-1,0.987945,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTGCTGTCGGGTG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTGCTGTCGGGTG...
3,3,gene_0,gene_9,-1,0.835812,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTGCTGTCGGGTG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTTCTGTCGGGTG...
4,4,gene_0,gene_15,-1,0.739491,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTGCTGTCGGGTG...,CGGGTCACGGCCAACTTCTGTCGGGAGCCGGCGAGCGGTCGCCCTC...,GCGGCAGGCGGGAGGGCGAGCGGGAGGGAAGGGAGTGGCGGGTGCG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTTCTGTCGGGTG...


In [303]:
# 生成prompts

prompts = []
for index, row in pair_CDS_exp.iterrows():
    
    for item in ['pair_A_CDS', 'pair_B_CDS']:
        
        ppt = str(Seq(row[item]).translate())
        
        for jtem in [ 'mRNA', 'UTR5','CDS', 'UTR3']:
            
            if jtem == 'mRNA':
                pairA_token = '_'.join([row[f'pair_A_UTR5'], row[f'pair_A_CDS'], row[f'pair_A_UTR3']  ])
                pairB_token = '_'.join([row[f'pair_B_UTR5'], row[f'pair_B_CDS'], row[f'pair_B_UTR3']  ])
                exact_diff = (pairA_token == pairB_token)
                roughly_diff = (pairA_token == pairB_token) 
            else:
                exact_diff = all([row[f'pair_A_{jtem}']==row[f'pair_B_{jtem}'] for tem in ['UTR5', 'UTR3','CDS'] if tem!=jtem])
                roughly_diff = all([row[f'pair_A_{tem}']==row[f'pair_B_{tem}'] or row[f'pair_A_{tem}'][4:-4]==row[f'pair_B_{tem}'][4:-4] for tem in ['UTR5', 'UTR3','CDS'] if tem!=jtem])
                pairA_token = row[f'pair_A_{jtem}']
                pairB_token = row[f'pair_B_{jtem}']
                
            label = row['pair_label']
            value = row['pair_diff']
            g_type=jtem
            origin = 'cds_exp'
            
            prompts.append([ppt, pairA_token, pairB_token, g_type, origin, label, value, exact_diff, roughly_diff])
            
            
            

In [304]:


CDS_prompts = pd.DataFrame(prompts, columns=['prompt', 'pair_A', 'pair_B', 'seq_type', 'origin', 'pair_label', 'pair_diff', 'exact_diff', 'roughly_diff'])

CDS_prompts_raw = CDS_prompts

CDS_prompts = CDS_prompts.drop_duplicates(subset=['prompt', 'pair_A', 'pair_B', 'seq_type'])

CDS_prompts = CDS_prompts[ 
                                    CDS_prompts['pair_A']  != CDS_prompts['pair_B']
                                    ]

CDS_prompts




,prompt,pair_A,pair_B,seq_type,origin,pair_label,pair_diff,exact_diff,roughly_diff
0,MRVTAPRTLILLLSGALALTETWAGSGGGGSGGIDEILDFFYGYQI...,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,mRNA,cds_exp,-1,0.831660,False,False
2,MRVTAPRTLILLLSGALALTETWAGSGGGGSGGIDEILDFFYGYQI...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTGCTGTCGGGTG...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTTCTGTCGGGTG...,CDS,cds_exp,-1,0.831660,False,True
8,MRVTAPRTLILLLSGALALTETWAGSGGGGSGGIDEILDFFYGYQI...,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,mRNA,cds_exp,-1,0.751856,False,False
9,MRVTAPRTLILLLSGALALTETWAGSGGGGSGGIDEILDFFYGYQI...,AAGCGGTACCGCGGTTGTTACGCTGCGTTTGTTCGTTGCTTTTTCG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,UTR5,cds_exp,-1,0.751856,False,False
10,MRVTAPRTLILLLSGALALTETWAGSGGGGSGGIDEILDFFYGYQI...,ATGCGGGTCACGGCCCCTAGAACACTGATTCTTCTGCTGTCGGGTG...,ATGAGAGTGACCGCCCCAAGGACCCTGATTCTCCTGCTGTCTGGAG...,CDS,cds_exp,-1,0.751856,False,False
...,...,...,...,...,...,...,...,...,...
1739,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GTGGTCCCAATTCTGGTCGAGCTCACGGACAGTTCTGTACCAAACC...,GCTGGTCCCAATTCTGGTCGAGCTCGGTCAACGGACAGTTCTGTCC...,UTR3,cds_exp,1,3.160315,False,False
1744,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGCTCA...,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGCCGAGCTCG...,mRNA,cds_exp,1,20.071805,False,False
1747,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GTGGTCCCAATTCTGGTCGAGCTCACGGACAGTTCTGTACCAAACC...,TCGAGCTCGGACAGTTCTGTACCGTGCGGGCCAAGGGCGAAGCGTC...,UTR3,cds_exp,1,20.071805,False,False
1752,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGCTCA...,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGCTCA...,mRNA,cds_exp,1,1.164065,False,False


##### pair_CDS2_expt

In [298]:


pair_CDS2_exp = pd.read_csv(f'data/interim/experiment_CDS2_pair_exp_cutoff{cutoff}.tsv', sep='\t')
pair_CDS2_exp = pair_CDS2_exp[pair_CDS2_exp['pair_label'] != 0]
pair_CDS2_exp.head()



,Unnamed: 0,pair_A,pair_B,pair_label,pair_diff,pair_A_UTR5,pair_A_UTR3,pair_A_CDS,pair_B_UTR5,pair_B_UTR3,pair_B_CDS
0,0,Neocura-Fluc_Exp11,Neocura-Fluc_Exp28,1,1.529703,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCATTCTATC...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAATATCAAGAAGGGCCCAGCCCCATTCTATC...
1,1,Neocura-Fluc_Exp11,Neocura-Fluc_Exp17,1,1.067821,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCATTCTATC...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATTAAGAAGGGCCCCGCTCCCTTCTATC...
2,2,Neocura-Fluc_Exp11,Neocura-Fluc_Exp23,-1,0.964888,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCATTCTATC...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCATTCTACC...
3,3,Neocura-Fluc_Exp11,Neocura-Fluc_Exp33,1,1.091166,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCATTCTATC...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCTTTCTACC...
4,4,Neocura-Fluc_Exp11,Neocura-Fluc_Exp04,1,1.040731,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCATTCTATC...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCCGCCCCTTTCTACC...


In [305]:
# 生成prompts

prompts = []
for index, row in pair_CDS2_exp.iterrows():
    
    for item in ['pair_A_CDS', 'pair_B_CDS']:
        
        ppt = str(Seq(row[item]).translate())
        
        for jtem in [ 'mRNA', 'UTR5','CDS', 'UTR3']:
            
            if jtem == 'mRNA':
                pairA_token = '_'.join([row[f'pair_A_UTR5'], row[f'pair_A_CDS'], row[f'pair_A_UTR3']  ])
                pairB_token = '_'.join([row[f'pair_B_UTR5'], row[f'pair_B_CDS'], row[f'pair_B_UTR3']  ])
                exact_diff = (pairA_token == pairB_token)
                roughly_diff = (pairA_token == pairB_token) 
            else:
                exact_diff = all([row[f'pair_A_{jtem}']==row[f'pair_B_{jtem}'] for tem in ['UTR5', 'UTR3','CDS'] if tem!=jtem])
                roughly_diff = all([row[f'pair_A_{tem}']==row[f'pair_B_{tem}'] or row[f'pair_A_{tem}'][4:-4]==row[f'pair_B_{tem}'][4:-4] for tem in ['UTR5', 'UTR3','CDS'] if tem!=jtem])
                pairA_token = row[f'pair_A_{jtem}']
                pairB_token = row[f'pair_B_{jtem}']
                
            label = row['pair_label']
            value = row['pair_diff']
            g_type=jtem
            origin = 'cds2_exp'
            
            prompts.append([ppt, pairA_token, pairB_token, g_type, origin, label, value, exact_diff, roughly_diff])
            
            
            


In [306]:


CDS2_prompts = pd.DataFrame(prompts, columns=['prompt', 'pair_A', 'pair_B', 'seq_type', 'origin', 'pair_label', 'pair_diff', 'exact_diff', 'roughly_diff'])

CDS2_prompts_raw = CDS2_prompts

CDS2_prompts = CDS2_prompts.drop_duplicates(subset=['prompt', 'pair_A', 'pair_B', 'seq_type'])

CDS2_prompts = CDS2_prompts[ 
                                    CDS2_prompts['pair_A']  != CDS2_prompts['pair_B']
                                    ]

CDS2_prompts





,prompt,pair_A,pair_B,seq_type,origin,pair_label,pair_diff,exact_diff,roughly_diff
0,MEDAKNIKKGPAPFYPLEDGTAGEQLHKAMKRYALVPGTIAFTDAH...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,mRNA,cds2_exp,1,1.529703,False,False
2,MEDAKNIKKGPAPFYPLEDGTAGEQLHKAMKRYALVPGTIAFTDAH...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCATTCTATC...,ATGGAGGACGCCAAGAATATCAAGAAGGGCCCAGCCCCATTCTATC...,CDS,cds2_exp,1,1.529703,False,True
8,MEDAKNIKKGPAPFYPLEDGTAGEQLHKAMKRYALVPGTIAFTDAH...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,mRNA,cds2_exp,1,1.067821,False,False
10,MEDAKNIKKGPAPFYPLEDGTAGEQLHKAMKRYALVPGTIAFTDAH...,ATGGAGGACGCCAAGAACATCAAGAAGGGCCCAGCCCCATTCTATC...,ATGGAGGACGCCAAGAACATTAAGAAGGGCCCCGCTCCCTTCTATC...,CDS,cds2_exp,1,1.067821,False,True
16,MEDAKNIKKGPAPFYPLEDGTAGEQLHKAMKRYALVPGTIAFTDAH...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,mRNA,cds2_exp,-1,0.964888,False,False
...,...,...,...,...,...,...,...,...,...
70472,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,AGGGGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGC...,mRNA,cds2_exp,1,1.500000,False,False
70480,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,AGGGGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGCCGAGC...,mRNA,cds2_exp,1,15.000000,False,False
70488,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,AGGGGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGC...,mRNA,cds2_exp,1,1.250000,False,False
70496,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,AGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,mRNA,cds2_exp,1,1.363636,False,False


##### merge all prompts

In [307]:

# raw prompots

# merge all prompts

merged_prompts_raw = pd.concat([UTR5_prompts_raw, UTR3_prompts_raw, CDS2_prompts_raw, CDS_prompts_raw, ])

merged_prompts_raw




,prompt,pair_A,pair_B,seq_type,origin,pair_label,pair_diff,exact_diff,roughly_diff
0,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,GGGAAATAAGAGAGAAAAGAAGAGTAAGAAGAAATATAAGACCCCG...,mRNA,utr5_exp,1,3.300000,False,False
1,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,GGGAAATAAGAGAGAAAAGAAGAGTAAGAAGAAATATAAGACCCCG...,UTR5,utr5_exp,1,3.300000,False,True
2,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,ATGGTGAGCAAGGGCGAGGAGCTGTTCACAGGCGTGGTGCCTATCC...,CDS,utr5_exp,1,3.300000,True,False
3,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,CTGGTACTGCATGCACGCAATGCTAGCTGCCCCTTTCCCGTCCTGG...,UTR3,utr5_exp,1,3.300000,True,False
4,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,GGGAAATAAGAGAGAAAAGAAGAGTAAGAAGAAATATAAGACCCCG...,mRNA,utr5_exp,1,3.300000,False,False
...,...,...,...,...,...,...,...,...,...
1755,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GTGGTCCCAATTCTGGTCGAGCTCACGGACAGTTCTGTACCAAACC...,GTGGTCCCAATTCTGGTCGAGCTCACGGACAGTTCTGTACCAAACC...,UTR3,cds_exp,1,1.164065,False,True
1756,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGCTCA...,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGCTCA...,mRNA,cds_exp,1,1.164065,False,False
1757,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGCTCA...,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGCTCA...,UTR5,cds_exp,1,1.164065,True,False
1758,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,ATGGTGAGCAAGGGCGAGGAGTTGTTCACCGGCGTGGTCCCAATTC...,CDS,cds_exp,1,1.164065,True,False


In [310]:

merged_prompts_raw.to_csv(f'data/rf_mRNA_small_cutoff{cutoff}_prompts_raw.csv', index=False)



In [308]:
# merge all prompts

# merged_prompts = pd.concat([UTR5_prompts, UTR3_prompts, CDS_prompts, cellines_prompts])


merged_prompts = pd.concat([UTR5_prompts, UTR3_prompts, CDS2_prompts, CDS_prompts])

merged_prompts = merged_prompts.drop_duplicates(subset=['prompt', 'pair_A', 'pair_B', 'seq_type'])

merged_prompts



,prompt,pair_A,pair_B,seq_type,origin,pair_label,pair_diff,exact_diff,roughly_diff
0,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,GGGAAATAAGAGAGAAAAGAAGAGTAAGAAGAAATATAAGACCCCG...,mRNA,utr5_exp,1,3.300000,False,False
1,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,GGGAAATAAGAGAGAAAAGAAGAGTAAGAAGAAATATAAGACCCCG...,UTR5,utr5_exp,1,3.300000,False,True
8,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,GGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,mRNA,utr5_exp,1,1.650000,False,False
9,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,GGGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,UTR5,utr5_exp,1,1.650000,False,True
16,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAAC...,GGGAATCAACACAGACACCAGACCACCAGCACCAGAACACACAGAC...,mRNA,utr5_exp,1,1.650000,False,False
...,...,...,...,...,...,...,...,...,...
1720,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGCTCA...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,mRNA,cds_exp,1,1.021720,False,False
1728,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGCTCA...,GGCAAGGGCGAGTTCACCGGCGTGGTCCCAATTCTGGTCGAGCTCG...,mRNA,cds_exp,1,3.415057,False,False
1736,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGCTCA...,GGCAAGGGCGAGTTCACCGGCGTGGTCCCAATTCTGGTCGAGCTCG...,mRNA,cds_exp,1,3.160315,False,False
1744,MVSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLK...,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGTCGAGCTCA...,GGCAAGGGCGAGTTCACCGGGCTGGTCCCAATTCTGGCCGAGCTCG...,mRNA,cds_exp,1,20.071805,False,False


In [156]:

merged_prompts['pair_label'].value_counts()
merged_prompts['seq_type'].value_counts()


UTR3    5428
UTR5    2242
CDS       86
Name: seq_type, dtype: int64

In [309]:

merged_prompts.to_csv(f'data/rf_mRNA_small_cutoff{cutoff}_prompts.csv', index=False)



##### 去重 & 分析

In [4]:

merged_prompts_raw = pd.read_csv(f'data/rf_mRNA_small_cutoff0_prompts_raw.csv')



In [5]:

merged_prompts_raw.shape


(197280, 9)

In [19]:


rank = 2

merged_prompts_ranked = merged_prompts_raw
merged_prompts_ranked.loc[merged_prompts_ranked['seq_type']=='mRNA', 'exact_diff'] = True # 预处理出错，此处矫正
merged_prompts_ranked.loc[merged_prompts_ranked['seq_type']=='mRNA', 'roughly_diff'] = True # 预处理出错，此处矫正

def ranked(x):
    
    if x > rank:
        return 1
    elif x <1/rank:
        return -1
    else:
        return 0
    
merged_prompts_ranked['pair_diff_label'] = merged_prompts_ranked['pair_diff'].apply(ranked) 

print(merged_prompts_ranked.shape)

# vote to get the concensus value
from scipy import stats

merged_prompts_ranked_grouped = merged_prompts_ranked.groupby(by=['prompt', 'pair_A', 'pair_B', 'seq_type']).agg(list)

print(merged_prompts_ranked_grouped.shape)

merged_prompts_ranked_grouped['pair_diff_label'] = merged_prompts_ranked_grouped['pair_diff_label'].apply(lambda x: stats.mode(x)[0][0])
merged_prompts_ranked_grouped['exact_diff'] = merged_prompts_ranked_grouped['exact_diff'].apply(lambda x: stats.mode(x)[0][0])

merged_prompts_ranked_grouped['roughly_diff'] = merged_prompts_ranked_grouped['roughly_diff'].apply(lambda x: stats.mode(x)[0][0])




(197280, 10)
(34251, 6)


In [20]:
#v过滤

merged_prompts_ranked_grouped_explode = merged_prompts_ranked_grouped[
    (merged_prompts_ranked_grouped['pair_diff_label']==-1) &
    (merged_prompts_ranked_grouped['roughly_diff']==True)
].explode(['origin', 'pair_label','pair_diff'])

# merged_prompts_ranked_grouped_explode = merged_prompts_ranked_grouped[
#     (merged_prompts_ranked_grouped['pair_diff_label']==-1)
# ].explode(['origin', 'pair_label', 'pair_diff'])



merged_prompts_ranked_grouped_explode = merged_prompts_ranked_grouped_explode.reset_index()

merged_prompts_ranked_grouped_explode = merged_prompts_ranked_grouped_explode[
        merged_prompts_ranked_grouped_explode['pair_A'] != merged_prompts_ranked_grouped_explode['pair_B'] 
                                                                                        ]

import numpy as np
     
# 去除重复对    
merged_prompts_ranked_grouped_explode['pairs'] = merged_prompts_ranked_grouped_explode.apply(lambda x: np.sort([x['prompt'], x['pair_A'], x['pair_B']]), axis=1)

merged_prompts_ranked_grouped_explode_dropped = merged_prompts_ranked_grouped_explode.drop_duplicates(subset= ['pairs'])

merged_prompts_ranked_grouped_explode_dropped




,prompt,pair_A,pair_B,seq_type,origin,pair_label,pair_diff,exact_diff,roughly_diff,pair_diff_label,pairs
0,MRVTAPRTLILLLSGALALTETWAGSGGGGSGGCSVDNCRLFISGI...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,mRNA,utr3_exp,-1,0.418709,True,True,-1,[AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCC...
1,MRVTAPRTLILLLSGALALTETWAGSGGGGSGGCSVDNCRLFISGI...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,mRNA,utr3_exp,-1,0.418709,True,True,-1,[AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCC...
2,MRVTAPRTLILLLSGALALTETWAGSGGGGSGGCSVDNCRLFISGI...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,mRNA,utr3_exp,-1,0.434997,True,True,-1,[AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCC...
3,MRVTAPRTLILLLSGALALTETWAGSGGGGSGGCSVDNCRLFISGI...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,mRNA,utr3_exp,-1,0.434997,True,True,-1,[AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCC...
4,MRVTAPRTLILLLSGALALTETWAGSGGGGSGGCSVDNCRLFISGI...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCG...,mRNA,utr3_exp,-1,0.4333,True,True,-1,[AATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCC...
...,...,...,...,...,...,...,...,...,...,...,...
13001,SRVQRVTDSLDTLLEISSNKQNHSYEYFMKQMNDARHGGWTTKMDW...,GGAGTCTCCCCTCGCCGCATCCACTCTCCGGCCGGCCGCCTGCCCG...,GGAAAGCGGTATCGGTGTGCGAGTCTTGCGGTTGTCTCTCTTCGCT...,mRNA,utr5_exp,-1,0.53103,True,True,-1,[GGAAAGCGGTATCGGTGTGCGAGTCTTGCGGTTGTCTCTCTTCGC...
13002,SRVQRVTDSLDTLLEISSNKQNHSYEYFMKQMNDARHGGWTTKMDW...,GGAGTCTCCCCTCGCCGCATCCACTCTCCGGCCGGCCGCCTGCCCG...,GGAAAGCGGTATCGGTGTGCGAGTCTTGCGGTTGTCTCTCTTCGCT...,mRNA,utr5_exp,-1,0.422966,True,True,-1,[GGAAAGCGGTATCGGTGTGCGAGTCTTGCGGTTGTCTCTCTTCGC...
13003,SRVQRVTDSLDTLLEISSNKQNHSYEYFMKQMNDARHGGWTTKMDW...,GGAGTCTCCCCTCGCCGCATCCACTCTCCGGCCGGCCGCCTGCCCG...,GGAAAGCGGTATCGGTGTGCGAGTCTTGCGGTTGTCTCTCTTCGCT...,mRNA,utr5_exp,-1,0.422966,True,True,-1,[GGAAAGCGGTATCGGTGTGCGAGTCTTGCGGTTGTCTCTCTTCGC...
13004,SRVQRVTDSLDTLLEISSNKQNHSYEYFMKQMNDARHGGWTTKMDW...,GGAGTCTCCCCTCGCCGCATCCACTCTCCGGCCGGCCGCCTGCCCG...,GGAAAGCGGTATCGGTGTGCGAGTCTTGCGGTTGTCTCTCTTCGCT...,mRNA,utr5_exp,-1,0.457813,True,True,-1,[GGAAAGCGGTATCGGTGTGCGAGTCTTGCGGTTGTCTCTCTTCGC...


In [21]:

merged_prompts_ranked_grouped_explode_dropped['seq_type'].value_counts()




mRNA    7598
UTR3    3514
UTR5    1142
CDS      744
Name: seq_type, dtype: int64

In [17]:

merged_prompts_ranked_grouped_explode_dropped['seq_type'].value_counts()



mRNA    11986
UTR3     4670
CDS      2320
UTR5     2028
Name: seq_type, dtype: int64

In [13]:
# analysis

merged_prompts_ranked_grouped_explode_dropped['seq_type'].value_counts()
# merged_prompts_ranked_grouped_explode_dropped['pair_diff_label'].value_counts()




mRNA    18596
UTR3     5684
UTR5     4524
CDS      4414
Name: seq_type, dtype: int64

In [22]:

rank

2

In [23]:
# save to file

merged_prompts_ranked_grouped_explode_dropped.to_csv(f'data/rf_mRNA_large_rank{rank}_roughly_diff_prompts.csv')




In [26]:
# load

rank = 1.5

merged_prompts_ranked_preference = pd.read_csv(f'data/rf_mRNA_large_rank{rank}_roughly_diff_prompts.csv')

merged_prompts_ranked_preference.shape



(21004, 12)

In [41]:

merged_prompts_ranked_preference['seq_type'].value_counts()

for col in ['prompt',	'pair_A',	'pair_B']:
    merged_prompts_ranked_preference[col+'_len'] = merged_prompts_ranked_preference[col].apply(len)
    
merged_prompts_ranked_preference['gen_seq'] = merged_prompts_ranked_preference['prompt'] + merged_prompts_ranked_preference['pair_A']
merged_prompts_ranked_preference['gen_seq_len'] = merged_prompts_ranked_preference['gen_seq'].apply(len)

merged_prompts_ranked_preference[[col for col in merged_prompts_ranked_preference.columns if '_len' in col]].describe().round(2)



,prompt_len,pair_A_len,pair_B_len,gen_seq_len
count,21004.00,21004.00,21004.00,21004.00
mean,305.44,870.07,853.57,1175.51
std,100.50,530.27,546.86,565.44
min,125.00,46.00,46.00,171.00
25%,286.00,292.00,214.00,586.00
50%,286.00,1035.00,1022.00,1314.00
75%,286.00,1221.00,1221.00,1507.00
max,811.00,2791.00,2791.00,3602.00
